In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv
/kaggle/working/__notebook__.ipynb


Il concetto alla base sta nel gestire ogni serie con il modello dedicato.
Questo perchè il dataset è costituito da chiavi composte da store_nbr, family, date e ognuno ha un proprio andamento stagionale e periodico.
Quindi il primo modello è lineare e viene allenato per individuare l'andamento tramite le trasformate di fourier
Il secondo è un semplice XBoost XGBRegressor. Ogni chiave ha il proprio modello ibrido.
In realtà occorrerebbe verificare che ogni serie presenti la stessa stagionalità e cicli.
Ci sono molte altre prove che potrebbero essere effettuate, con i lag e capendo come utilizzare i dati aggiuntivi
Nella timeseries education alcune prove con deep learning che però utilizzano una finestra di 6 giorni precedenti per predire il giorno successivo

In [2]:
import warnings
from statsmodels.tsa.deterministic import DeterministicProcess

warnings.filterwarnings('ignore')

def getDfs():
    holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv', parse_dates=True)
    oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates=True)
    stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv', parse_dates=True)
    train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', parse_dates=True, index_col='id')
    test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv', parse_dates=True, index_col='id')
    transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv', parse_dates=True)
    
    return holidays, oil, stores, train, test, transactions


def getX():
    holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv', parse_dates=True)
    oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates=True)
    stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv', parse_dates=True)
    train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', parse_dates=True, index_col='id')
    test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv', parse_dates=True, index_col='id')
    transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv', parse_dates=True)
    
    # Unisco train e test per poter mergiare le altre colonne - Inferenza
    X = pd.concat([train, test], axis=0)
    # transactions# Provo ad inserire il transactions lag_1 dello store
    # visto che c'è correlazione con il giorno precedente normalmente
    #transactions['transactions_lag1'] = transactions['transactions'].shift(0).fillna(0)
    #X = train.copy()
    X = pd.merge(X, stores[['store_nbr', 'type', 'cluster']], on='store_nbr', how='left')
    #X = pd.merge(X, transactions[['date', 'store_nbr', 'transactions']], on=['date','store_nbr'], how='left')
    X = pd.merge(X, oil[['date', 'dcoilwtico']], on='date', how='left')
    #X = pd.merge(X, holidays[['date', 'type', 'transferred']], on='date', how='left')    
    X['date'] = pd.to_datetime(X['date'])

    #Proviamo con 
    X = X[X['date']>pd.to_datetime('2015-12-31')]

    return X, test

def getTrainTest():
    train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', parse_dates=True, index_col='id')
    test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv', parse_dates=True, index_col='id')
    
    X = train.copy()
    X['date'] = pd.to_datetime(X['date'])
    test['date'] = pd.to_datetime(test['date'])

    # Prendiamo solamente le date oltre 2015
    #X = X[X['date']>pd.to_datetime('2015-12-31')]
    #test =test[test['date']>pd.to_datetime('2015-12-31')]
    
    return X, test

def getXLagged():
    train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', parse_dates=True, index_col='id')
    test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv', parse_dates=True, index_col='id')
    
    train['Lag_1'] = train['sales'].shift(1).fillna(0)

    return train, test

def getGrouped(family, target, X=None):
    if X is None:
        X, _ = getX()
    XX = X[X['family']==family].groupby('date')[target].sum().reset_index()
    X['date'] = pd.to_datetime(X['date'])

    return XX

from sklearn.preprocessing import LabelEncoder
def labelEncodeFeatures(X, features=None):
    if features is None:
        features = [feat for feat in X.columns if X[feat].dtype=='object']
    for feature in features:
        label_encoder = LabelEncoder()
        X[feature] = label_encoder.fit_transform(X[feature])
    return label_encoder

In [3]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
'''
def addFourierToTrain(series):
    series = series.sort_values(by='date').reset_index(drop=True)
    # Imposta la colonna 'date' come indice
    series = series.set_index('date')
    # Devo accertarmi che l'indice non abbia buchi
    #series.index.freq = pd.DateOffset(days=1)

    fourier = CalendarFourier(freq='M', order=4)
    
    dp = DeterministicProcess(
        constant=True,
        index=series.index,
        order=1,
        seasonal=True,
        drop=True,
        additional_terms=[fourier],
    )
    
    X_sample_fourier = dp.in_sample()  
    series = pd.concat([series,X_sample_fourier], axis=1)
    return series
'''
def addCrhistmas(series):
    existing_value = series[series.date == pd.to_datetime('2013-12-26')].copy()
    existing_value.date = pd.to_datetime('2013-12-25')
    existing_value.sales = 0
    series = pd.concat([series, existing_value], ignore_index=True)
    existing_value = series[series.date == pd.to_datetime('2014-12-26')].copy()
    existing_value.date = pd.to_datetime('2014-12-25')
    existing_value.sales = 0
    series = pd.concat([series, existing_value], ignore_index=True)
    existing_value = series[series.date == pd.to_datetime('2015-12-26')].copy()
    existing_value.date = pd.to_datetime('2015-12-25')
    existing_value.sales = 0
    series = pd.concat([series, existing_value], ignore_index=True)
    existing_value = series[series.date == pd.to_datetime('2016-12-26')].copy()
    existing_value.date = pd.to_datetime('2016-12-25')
    existing_value.sales = 0
    series = pd.concat([series, existing_value], ignore_index=True)
    return series

# correggo i NaN di dcoilwtico prendendo il valore NaN più vicino
def fill_nan_with_nearest(arr):
    filled_arr = arr.copy()
    mask = np.isnan(filled_arr)    
    for i in range(1, len(filled_arr)):
        if mask[i]:
            filled_arr[i] = filled_arr[i-1]  # Sostituisci con il valore precedente non NaN

    for i in range(len(filled_arr) - 2, -1, -1):
        if mask[i]:
            filled_arr[i] = filled_arr[i+1]  # Sostituisci con il valore successivo non NaN

    return filled_arr

# Se voglio utilizzare i tempi precedenti come input devo creare i lag da t0 a t_n
def createLags(X, target, n_lags):
    lags = {f'lag_{i}': X[target].shift(i) for i in range(1,n_lags+1)}
    lags_df = pd.DataFrame(lags)
    X = pd.concat([X, lags_df], axis=1)
    return X
'''    
def addFeaturesToTrain(series):
    holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv', parse_dates=True)
    oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates=True)
    holidays['date'] = pd.to_datetime(holidays['date'])
    oil['date'] = pd.to_datetime(oil['date'])
    #series = series.reset_index()
    series = pd.merge(series, oil[['date', 'dcoilwtico']], on='date', how='left')
    series = pd.merge(series, holidays[['date', 'type', 'transferred']], on='date', how='left')    
    labelEncodeFeatures(series)
    return series
'''
def CreateSeriesDf():
    grouping = 'LIQUOR,WINE,BEER'
    series = getGrouped(grouping, 'sales')
    series = addCrhistmas(series)
        
    # Decidiamo i dati da passare al modello.
    # Sicuramente i dati periodici
    # Vediamo gli altri dati delle tabelle collegabili
    series = addFourierToTrain(series)
    series = addFeaturesToTrain(series)
    series = createLags(series, 'sales', 6)
        
    return series

Preparo un modello per ogni chiave

In [4]:
# modello con deterministic process
#series = getGrouped('PREPARED FOODS', 'sales')
def step1():
    X,_ = getX()
    X = X.reset_index()    
    series = X.copy()
    print(f'step1: 0')
    unique_couples =  series[['store_nbr', 'family']].drop_duplicates()

    final_series = None
    print(f'step1: 1')

    for index, row in unique_couples.iterrows():
        
        store = row.store_nbr
        family = row.family
        print('\r' + f'step1 addChristmas for key: {store}_{family}', end='', flush=True)
        xmas = addCrhistmas(series[(series['store_nbr']==store) & (series['family']==family)])
        final_series = pd.concat([final_series,xmas], axis=0, ignore_index=True)

    final_series = final_series.sort_values(['store_nbr','family','date'])

    return final_series

In [5]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
def step2():
    series = step1()

    def getDpFourier(Xs):
        fourier = CalendarFourier(freq='M', order=4)
        results = []
        Xs = Xs.set_index(['store_nbr', 'family', 'date'])
        # Per il modello lineare passo solamente le features che servono a stabilire l'andamento sinusoidale
        coppie_uniche = Xs.index.unique()
        print(f'getDpFourier: index unique')
        
        # Creiamo un set per memorizzare le coppie uniche
        unique_pairs = set()
        # Iteriamo su tutte le coppie nell'indice
        for store, family, _ in coppie_uniche:
            unique_pairs.add((store, family))

        print(f'getDpFourier: 0')
        for store, family in unique_pairs:
            key_model = str(store) + '_' + family
            print('\r' + f'set Dp Fourier for key_model: {key_model}', end='', flush=True)
            store_data = Xs.loc[store]
            family_data = store_data.loc[family]
            
            dp = DeterministicProcess(
                index=family_data.index,  # Usa l'index
                constant=True,     
                order=1,           
                seasonal=True,     # stagionalità settimanale gestita con gli indicators
                additional_terms=[fourier],  # stagionalità mensile (fourier)
                drop=True,        
            )
            sample = dp.in_sample()  
            
            # Ripristina l'indice originario
            sample.index = pd.MultiIndex.from_product([[store], [family], sample.index], names=['store_nbr', 'family', 'date'])            
            # Aggiungi il risultato
            results.append(sample)
    
        # Unisci i risultati in un DataFrame finale
        return pd.concat(results)
    
    X_sample_fourier = getDpFourier(series)
    return X_sample_fourier, series

In [6]:
'''
import seaborn as sns

def getMinReasonableDates():
    train, _ =  getX()
    train['sales']=train['sales'].fillna(0)
    train['dcoilwtico']=train['dcoilwtico'].fillna(0)

    minReasonableDates={}
    # Itero su tutte le chiavi store-family per capire da quale data è meglio considerare i dati per il training
    unique_stores = train['store_nbr'].unique()
    unique_families = train['family'].unique()
    firstDate = train['date'].min()
    for store in unique_stores:
        for family in unique_families:
            key = f'{store}_{family}'
            #print(key)
            slice = ((train['store_nbr']==store) & (train['family']==family))
            df = train.loc[slice].copy()
            # 1. Definire il momento a partire dal quale i valori sono sempre maggiori di zero
            first_non_zero_index = df[df['sales'] > 0].index.min()
            
            if first_non_zero_index is not None and pd.notna(first_non_zero_index):
                #print(f'first_non_zero_index:{first_non_zero_index}')
                date_over_zero = df.loc[first_non_zero_index]['date']
            
                # 2. Definire il primo ID dopo il quale il valore assume zero per non più di n giorni consecutivi
                n = 20  # Numero massimo di giorni consecutivi per cui il valore può essere zero
                df['is_zero'] = df['sales'] == 0
                
                # Creazione della colonna per le sequenze di zeri
                df['group'] = (df['is_zero'] != df['is_zero'].shift()).cumsum()
                
                # Filtrare i gruppi di zeri e contare la lunghezza delle sequenze di zero
                zero_groups = df[df['is_zero']].groupby('group').agg(start=('date', 'min'), end=('date', 'max'), length=('sales', 'count'))
                
                # Trova i gruppi che hanno lunghezza maggiore di n
                long_zero_sequences = zero_groups[zero_groups['length'] > n]
                
                # Trova il primo ID dopo l'ultima sequenza di zero
                if not long_zero_sequences.empty:
                    last_long_zero_sequence_end = long_zero_sequences['end'].max()
                    
                    # L'ID di partenza è il giorno successivo a massimo del gruppo di zeri
                    starting_point = last_long_zero_sequence_end + pd.Timedelta(days=1)
                    minReasonableDates[key] = starting_point
                else:
                    #print(f"Non ci sono sequenze di zeri maggiori di {n} giorni consecutivi per {key}")
                    minReasonableDates[key] = firstDate
    print('end of getMinReasonableDates()')
    return minReasonableDates
    
#sns.scatterplot(data=, x='date', y='sales', hue='family')

'''

'\nimport seaborn as sns\n\ndef getMinReasonableDates():\n    train, _ =  getX()\n    train[\'sales\']=train[\'sales\'].fillna(0)\n    train[\'dcoilwtico\']=train[\'dcoilwtico\'].fillna(0)\n\n    minReasonableDates={}\n    # Itero su tutte le chiavi store-family per capire da quale data è meglio considerare i dati per il training\n    unique_stores = train[\'store_nbr\'].unique()\n    unique_families = train[\'family\'].unique()\n    firstDate = train[\'date\'].min()\n    for store in unique_stores:\n        for family in unique_families:\n            key = f\'{store}_{family}\'\n            #print(key)\n            slice = ((train[\'store_nbr\']==store) & (train[\'family\']==family))\n            df = train.loc[slice].copy()\n            # 1. Definire il momento a partire dal quale i valori sono sempre maggiori di zero\n            first_non_zero_index = df[df[\'sales\'] > 0].index.min()\n            \n            if first_non_zero_index is not None and pd.notna(first_non_zero_index):

In [7]:
import pandas as pd

def getMinReasonableDates():
    train, _ = getX()
    train['sales'] = train['sales'].fillna(0)
    train['dcoilwtico'] = train['dcoilwtico'].fillna(0)

    minReasonableDates = {}
    unique_couples =  train[['store_nbr', 'family']].drop_duplicates()

    firstDate = train['date'].min()

    for index, row in unique_couples.iterrows():
        store = row.store_nbr
        family = row.family
        key = f'{store}_{family}'
        print('\r' + f'key: {key}', end='', flush=True)
        
        slice = (train['store_nbr'] == store) & (train['family'] == family)
        df = train.loc[slice].copy()
        
        # 1. Trova il primo index con sales > 0
        first_non_zero_index = df[df['sales'] > 0].index.min()
        
        if first_non_zero_index is not None and pd.notna(first_non_zero_index):
            date_over_zero = df.loc[first_non_zero_index]['date']
            
            # 2. Identificare le sequenze di zeri
            n = 45  # Numero massimo di giorni consecutivi per cui il valore può essere zero
            df['is_zero'] = df['sales'] == 0
            
            # Creazione della colonna per le sequenze di zeri
            df['group'] = (df['is_zero'] != df['is_zero'].shift()).cumsum()
            
            # Filtrare i gruppi di zeri
            zero_groups = df[df['is_zero']].groupby('group').agg(start=('date', 'min'), end=('date', 'max'), length=('sales', 'count'))
            
            # Trova i gruppi che hanno lunghezza maggiore di n
            long_zero_sequences = zero_groups[zero_groups['length'] > n]
            
            if not long_zero_sequences.empty:
                last_long_zero_sequence_end = long_zero_sequences['end'].max()
                
                # L'ID di partenza è il giorno successivo a massimo del gruppo di zeri
                starting_point = last_long_zero_sequence_end + pd.Timedelta(days=1)
                minReasonableDates[key] = starting_point
            else:
                # Controlla se ci sono zero consecutivi dopo la "data_over_zero"
                following_zeros = df.loc[first_non_zero_index:, 'sales'] == 0
                
                if following_zeros.all():
                    # Se sono tutti zero, prendi la penultima data valida
                    last_valid_date_index = df[df['sales'] > 0].index.max()
                    if last_valid_date_index is not None:
                        minReasonableDates[key] = df.loc[last_valid_date_index]['date']
                    else:
                        minReasonableDates[key] = firstDate
                else:
                    minReasonableDates[key] = date_over_zero
        else:
            minReasonableDates[key] = firstDate
    
    print('end of getMinReasonableDates()')
    return minReasonableDates

In [8]:
### RISULTATO DEL getMinReasonableDates
def getreasonabledates():
    reasonabledates = {"1_AUTOMOTIVE":"2013-01-02T00:00:00","1_BABY CARE":"2013-01-01T00:00:00","1_BEAUTY":"2013-01-02T00:00:00","1_BEVERAGES":"2013-01-02T00:00:00","1_BOOKS":"2016-10-13T00:00:00","1_BREAD/BAKERY":"2013-01-02T00:00:00","1_CELEBRATION":"2015-06-01T00:00:00","1_CLEANING":"2013-01-02T00:00:00","1_DAIRY":"2013-01-02T00:00:00","1_DELI":"2013-01-02T00:00:00","1_EGGS":"2013-01-02T00:00:00","1_FROZEN FOODS":"2013-01-02T00:00:00","1_GROCERY I":"2013-01-02T00:00:00","1_GROCERY II":"2013-01-02T00:00:00","1_HARDWARE":"2013-01-02T00:00:00","1_HOME AND KITCHEN I":"2014-07-01T00:00:00","1_HOME AND KITCHEN II":"2014-07-01T00:00:00","1_HOME APPLIANCES":"2017-01-19T00:00:00","1_HOME CARE":"2015-05-04T00:00:00","1_LADIESWEAR":"2015-06-01T00:00:00","1_LAWN AND GARDEN":"2013-01-02T00:00:00","1_LINGERIE":"2013-01-02T00:00:00","1_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","1_MAGAZINES":"2015-04-10T00:00:00","1_MEATS":"2013-01-02T00:00:00","1_PERSONAL CARE":"2013-01-02T00:00:00","1_PET SUPPLIES":"2015-06-01T00:00:00","1_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","1_POULTRY":"2013-01-02T00:00:00","1_PREPARED FOODS":"2013-01-02T00:00:00","1_PRODUCE":"2013-03-16T00:00:00","1_SCHOOL AND OFFICE SUPPLIES":"2017-09-01T00:00:00","1_SEAFOOD":"2013-01-02T00:00:00","10_AUTOMOTIVE":"2013-01-02T00:00:00","10_BABY CARE":"2017-09-01T00:00:00","10_BEAUTY":"2013-01-02T00:00:00","10_BEVERAGES":"2013-01-02T00:00:00","10_BOOKS":"2013-01-01T00:00:00","10_BREAD/BAKERY":"2013-01-02T00:00:00","10_CELEBRATION":"2015-06-01T00:00:00","10_CLEANING":"2013-01-02T00:00:00","10_DAIRY":"2013-01-02T00:00:00","10_DELI":"2013-01-02T00:00:00","10_EGGS":"2013-01-02T00:00:00","10_FROZEN FOODS":"2013-01-02T00:00:00","10_GROCERY I":"2013-01-02T00:00:00","10_GROCERY II":"2013-01-02T00:00:00","10_HARDWARE":"2017-01-06T00:00:00","10_HOME AND KITCHEN I":"2014-07-01T00:00:00","10_HOME AND KITCHEN II":"2014-07-01T00:00:00","10_HOME APPLIANCES":"2016-12-30T00:00:00","10_HOME CARE":"2015-05-04T00:00:00","10_LADIESWEAR":"2015-06-01T00:00:00","10_LAWN AND GARDEN":"2017-09-01T00:00:00","10_LINGERIE":"2013-01-02T00:00:00","10_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","10_MAGAZINES":"2015-07-11T00:00:00","10_MEATS":"2013-01-02T00:00:00","10_PERSONAL CARE":"2013-01-02T00:00:00","10_PET SUPPLIES":"2015-06-05T00:00:00","10_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","10_POULTRY":"2013-01-02T00:00:00","10_PREPARED FOODS":"2013-01-02T00:00:00","10_PRODUCE":"2015-06-01T00:00:00","10_SCHOOL AND OFFICE SUPPLIES":"2017-08-08T00:00:00","10_SEAFOOD":"2013-09-23T00:00:00","11_AUTOMOTIVE":"2013-01-02T00:00:00","11_BABY CARE":"2015-12-12T00:00:00","11_BEAUTY":"2013-01-02T00:00:00","11_BEVERAGES":"2013-01-02T00:00:00","11_BOOKS":"2013-01-01T00:00:00","11_BREAD/BAKERY":"2013-01-02T00:00:00","11_CELEBRATION":"2015-06-01T00:00:00","11_CLEANING":"2013-01-02T00:00:00","11_DAIRY":"2013-01-02T00:00:00","11_DELI":"2013-01-02T00:00:00","11_EGGS":"2013-01-02T00:00:00","11_FROZEN FOODS":"2013-01-02T00:00:00","11_GROCERY I":"2013-01-02T00:00:00","11_GROCERY II":"2013-01-02T00:00:00","11_HARDWARE":"2013-01-02T00:00:00","11_HOME AND KITCHEN I":"2014-07-01T00:00:00","11_HOME AND KITCHEN II":"2014-07-01T00:00:00","11_HOME APPLIANCES":"2016-09-07T00:00:00","11_HOME CARE":"2015-05-04T00:00:00","11_LADIESWEAR":"2015-06-01T00:00:00","11_LAWN AND GARDEN":"2013-01-05T00:00:00","11_LINGERIE":"2013-01-02T00:00:00","11_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","11_MAGAZINES":"2015-04-11T00:00:00","11_MEATS":"2013-01-02T00:00:00","11_PERSONAL CARE":"2013-01-02T00:00:00","11_PET SUPPLIES":"2015-06-01T00:00:00","11_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","11_POULTRY":"2013-01-02T00:00:00","11_PREPARED FOODS":"2013-01-02T00:00:00","11_PRODUCE":"2015-06-01T00:00:00","11_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","11_SEAFOOD":"2013-01-02T00:00:00","12_AUTOMOTIVE":"2015-05-29T00:00:00","12_BABY CARE":"2017-09-01T00:00:00","12_BEAUTY":"2015-05-29T00:00:00","12_BEVERAGES":"2015-05-29T00:00:00","12_BOOKS":"2013-01-01T00:00:00","12_BREAD/BAKERY":"2015-05-29T00:00:00","12_CELEBRATION":"2015-06-01T00:00:00","12_CLEANING":"2015-05-29T00:00:00","12_DAIRY":"2015-05-29T00:00:00","12_DELI":"2015-05-29T00:00:00","12_EGGS":"2015-05-29T00:00:00","12_FROZEN FOODS":"2015-05-29T00:00:00","12_GROCERY I":"2015-05-29T00:00:00","12_GROCERY II":"2015-05-29T00:00:00","12_HARDWARE":"2015-05-29T00:00:00","12_HOME AND KITCHEN I":"2015-05-29T00:00:00","12_HOME AND KITCHEN II":"2015-05-29T00:00:00","12_HOME APPLIANCES":"2016-12-17T00:00:00","12_HOME CARE":"2015-05-29T00:00:00","12_LADIESWEAR":"2017-04-27T00:00:00","12_LAWN AND GARDEN":"2017-09-01T00:00:00","12_LINGERIE":"2015-05-29T00:00:00","12_LIQUOR,WINE,BEER":"2015-05-29T00:00:00","12_MAGAZINES":"2015-05-29T00:00:00","12_MEATS":"2015-05-29T00:00:00","12_PERSONAL CARE":"2015-05-29T00:00:00","12_PET SUPPLIES":"2015-06-02T00:00:00","12_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","12_POULTRY":"2015-05-29T00:00:00","12_PREPARED FOODS":"2015-05-29T00:00:00","12_PRODUCE":"2015-06-01T00:00:00","12_SCHOOL AND OFFICE SUPPLIES":"2017-03-18T00:00:00","12_SEAFOOD":"2015-05-29T00:00:00","13_AUTOMOTIVE":"2013-01-02T00:00:00","13_BABY CARE":"2013-01-01T00:00:00","13_BEAUTY":"2013-01-02T00:00:00","13_BEVERAGES":"2013-01-02T00:00:00","13_BOOKS":"2013-01-01T00:00:00","13_BREAD/BAKERY":"2013-01-02T00:00:00","13_CELEBRATION":"2015-06-01T00:00:00","13_CLEANING":"2013-01-02T00:00:00","13_DAIRY":"2013-01-02T00:00:00","13_DELI":"2013-01-02T00:00:00","13_EGGS":"2013-01-02T00:00:00","13_FROZEN FOODS":"2013-01-02T00:00:00","13_GROCERY I":"2013-01-02T00:00:00","13_GROCERY II":"2013-01-02T00:00:00","13_HARDWARE":"2013-01-05T00:00:00","13_HOME AND KITCHEN I":"2014-07-01T00:00:00","13_HOME AND KITCHEN II":"2014-07-01T00:00:00","13_HOME APPLIANCES":"2017-01-23T00:00:00","13_HOME CARE":"2015-05-04T00:00:00","13_LADIESWEAR":"2015-06-01T00:00:00","13_LAWN AND GARDEN":"2017-09-01T00:00:00","13_LINGERIE":"2013-01-02T00:00:00","13_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","13_MAGAZINES":"2015-04-07T00:00:00","13_MEATS":"2013-01-02T00:00:00","13_PERSONAL CARE":"2013-01-02T00:00:00","13_PET SUPPLIES":"2015-06-22T00:00:00","13_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","13_POULTRY":"2013-01-02T00:00:00","13_PREPARED FOODS":"2013-01-02T00:00:00","13_PRODUCE":"2015-06-01T00:00:00","13_SCHOOL AND OFFICE SUPPLIES":"2017-08-08T00:00:00","13_SEAFOOD":"2014-08-25T00:00:00","14_AUTOMOTIVE":"2013-01-02T00:00:00","14_BABY CARE":"2017-09-01T00:00:00","14_BEAUTY":"2013-01-02T00:00:00","14_BEVERAGES":"2013-01-02T00:00:00","14_BOOKS":"2013-01-01T00:00:00","14_BREAD/BAKERY":"2013-01-02T00:00:00","14_CELEBRATION":"2015-06-01T00:00:00","14_CLEANING":"2013-01-02T00:00:00","14_DAIRY":"2013-01-02T00:00:00","14_DELI":"2013-01-02T00:00:00","14_EGGS":"2013-01-02T00:00:00","14_FROZEN FOODS":"2013-01-02T00:00:00","14_GROCERY I":"2013-01-02T00:00:00","14_GROCERY II":"2013-01-02T00:00:00","14_HARDWARE":"2015-08-18T00:00:00","14_HOME AND KITCHEN I":"2014-07-01T00:00:00","14_HOME AND KITCHEN II":"2014-07-01T00:00:00","14_HOME APPLIANCES":"2016-05-01T00:00:00","14_HOME CARE":"2015-05-04T00:00:00","14_LADIESWEAR":"2017-05-04T00:00:00","14_LAWN AND GARDEN":"2013-01-01T00:00:00","14_LINGERIE":"2013-01-02T00:00:00","14_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","14_MAGAZINES":"2015-04-03T00:00:00","14_MEATS":"2013-01-02T00:00:00","14_PERSONAL CARE":"2013-01-02T00:00:00","14_PET SUPPLIES":"2015-06-02T00:00:00","14_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","14_POULTRY":"2013-01-02T00:00:00","14_PREPARED FOODS":"2013-01-02T00:00:00","14_PRODUCE":"2015-06-01T00:00:00","14_SCHOOL AND OFFICE SUPPLIES":"2016-07-19T00:00:00","14_SEAFOOD":"2014-09-11T00:00:00","15_AUTOMOTIVE":"2013-01-02T00:00:00","15_BABY CARE":"2015-12-07T00:00:00","15_BEAUTY":"2013-01-02T00:00:00","15_BEVERAGES":"2013-01-02T00:00:00","15_BOOKS":"2013-01-01T00:00:00","15_BREAD/BAKERY":"2013-01-02T00:00:00","15_CELEBRATION":"2015-06-01T00:00:00","15_CLEANING":"2013-01-02T00:00:00","15_DAIRY":"2013-01-02T00:00:00","15_DELI":"2013-01-02T00:00:00","15_EGGS":"2013-01-02T00:00:00","15_FROZEN FOODS":"2013-01-02T00:00:00","15_GROCERY I":"2013-01-02T00:00:00","15_GROCERY II":"2013-01-02T00:00:00","15_HARDWARE":"2013-01-02T00:00:00","15_HOME AND KITCHEN I":"2014-07-01T00:00:00","15_HOME AND KITCHEN II":"2014-07-01T00:00:00","15_HOME APPLIANCES":"2017-02-05T00:00:00","15_HOME CARE":"2015-05-04T00:00:00","15_LADIESWEAR":"2015-06-01T00:00:00","15_LAWN AND GARDEN":"2017-09-01T00:00:00","15_LINGERIE":"2013-01-02T00:00:00","15_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","15_MAGAZINES":"2015-04-22T00:00:00","15_MEATS":"2013-01-02T00:00:00","15_PERSONAL CARE":"2013-01-02T00:00:00","15_PET SUPPLIES":"2015-06-01T00:00:00","15_PLAYERS AND ELECTRONICS":"2015-06-02T00:00:00","15_POULTRY":"2013-01-03T00:00:00","15_PREPARED FOODS":"2013-01-02T00:00:00","15_PRODUCE":"2015-06-01T00:00:00","15_SCHOOL AND OFFICE SUPPLIES":"2017-08-09T00:00:00","15_SEAFOOD":"2013-09-26T00:00:00","16_AUTOMOTIVE":"2013-01-02T00:00:00","16_BABY CARE":"2017-07-27T00:00:00","16_BEAUTY":"2013-01-03T00:00:00","16_BEVERAGES":"2013-01-02T00:00:00","16_BOOKS":"2013-01-01T00:00:00","16_BREAD/BAKERY":"2013-01-02T00:00:00","16_CELEBRATION":"2015-06-01T00:00:00","16_CLEANING":"2013-01-02T00:00:00","16_DAIRY":"2013-01-02T00:00:00","16_DELI":"2013-01-02T00:00:00","16_EGGS":"2013-01-02T00:00:00","16_FROZEN FOODS":"2013-01-02T00:00:00","16_GROCERY I":"2013-01-02T00:00:00","16_GROCERY II":"2013-01-02T00:00:00","16_HARDWARE":"2013-01-03T00:00:00","16_HOME AND KITCHEN I":"2014-07-01T00:00:00","16_HOME AND KITCHEN II":"2014-07-01T00:00:00","16_HOME APPLIANCES":"2017-02-08T00:00:00","16_HOME CARE":"2015-05-04T00:00:00","16_LADIESWEAR":"2013-01-01T00:00:00","16_LAWN AND GARDEN":"2017-09-01T00:00:00","16_LINGERIE":"2013-01-02T00:00:00","16_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","16_MAGAZINES":"2015-04-20T00:00:00","16_MEATS":"2013-01-02T00:00:00","16_PERSONAL CARE":"2013-01-02T00:00:00","16_PET SUPPLIES":"2015-10-11T00:00:00","16_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","16_POULTRY":"2013-01-02T00:00:00","16_PREPARED FOODS":"2013-01-02T00:00:00","16_PRODUCE":"2015-06-01T00:00:00","16_SCHOOL AND OFFICE SUPPLIES":"2017-09-01T00:00:00","16_SEAFOOD":"2013-09-23T00:00:00","17_AUTOMOTIVE":"2013-01-02T00:00:00","17_BABY CARE":"2016-03-04T00:00:00","17_BEAUTY":"2013-01-02T00:00:00","17_BEVERAGES":"2013-01-02T00:00:00","17_BOOKS":"2013-01-01T00:00:00","17_BREAD/BAKERY":"2013-01-02T00:00:00","17_CELEBRATION":"2015-06-01T00:00:00","17_CLEANING":"2013-01-02T00:00:00","17_DAIRY":"2013-01-02T00:00:00","17_DELI":"2013-01-02T00:00:00","17_EGGS":"2013-01-02T00:00:00","17_FROZEN FOODS":"2013-01-02T00:00:00","17_GROCERY I":"2013-01-02T00:00:00","17_GROCERY II":"2013-01-02T00:00:00","17_HARDWARE":"2016-03-01T00:00:00","17_HOME AND KITCHEN I":"2014-07-01T00:00:00","17_HOME AND KITCHEN II":"2014-07-01T00:00:00","17_HOME APPLIANCES":"2016-12-14T00:00:00","17_HOME CARE":"2015-05-04T00:00:00","17_LADIESWEAR":"2017-03-26T00:00:00","17_LAWN AND GARDEN":"2017-09-01T00:00:00","17_LINGERIE":"2013-01-02T00:00:00","17_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","17_MAGAZINES":"2015-04-05T00:00:00","17_MEATS":"2013-01-02T00:00:00","17_PERSONAL CARE":"2013-01-02T00:00:00","17_PET SUPPLIES":"2015-06-01T00:00:00","17_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","17_POULTRY":"2013-01-02T00:00:00","17_PREPARED FOODS":"2013-01-02T00:00:00","17_PRODUCE":"2015-06-01T00:00:00","17_SCHOOL AND OFFICE SUPPLIES":"2017-03-13T00:00:00","17_SEAFOOD":"2013-09-21T00:00:00","18_AUTOMOTIVE":"2016-12-03T00:00:00","18_BABY CARE":"2016-12-03T00:00:00","18_BEAUTY":"2016-12-03T00:00:00","18_BEVERAGES":"2016-12-03T00:00:00","18_BOOKS":"2013-01-01T00:00:00","18_BREAD/BAKERY":"2016-12-03T00:00:00","18_CELEBRATION":"2016-12-03T00:00:00","18_CLEANING":"2016-12-03T00:00:00","18_DAIRY":"2016-12-03T00:00:00","18_DELI":"2016-12-03T00:00:00","18_EGGS":"2016-12-03T00:00:00","18_FROZEN FOODS":"2016-12-03T00:00:00","18_GROCERY I":"2016-12-03T00:00:00","18_GROCERY II":"2016-12-03T00:00:00","18_HARDWARE":"2016-12-03T00:00:00","18_HOME AND KITCHEN I":"2016-12-03T00:00:00","18_HOME AND KITCHEN II":"2016-12-03T00:00:00","18_HOME APPLIANCES":"2017-02-11T00:00:00","18_HOME CARE":"2016-12-03T00:00:00","18_LADIESWEAR":"2016-12-03T00:00:00","18_LAWN AND GARDEN":"2016-12-03T00:00:00","18_LINGERIE":"2016-12-03T00:00:00","18_LIQUOR,WINE,BEER":"2016-12-03T00:00:00","18_MAGAZINES":"2016-12-03T00:00:00","18_MEATS":"2016-12-03T00:00:00","18_PERSONAL CARE":"2016-12-03T00:00:00","18_PET SUPPLIES":"2016-12-03T00:00:00","18_PLAYERS AND ELECTRONICS":"2016-12-03T00:00:00","18_POULTRY":"2016-12-03T00:00:00","18_PREPARED FOODS":"2016-12-03T00:00:00","18_PRODUCE":"2016-12-03T00:00:00","18_SCHOOL AND OFFICE SUPPLIES":"2016-12-03T00:00:00","18_SEAFOOD":"2016-12-03T00:00:00","19_AUTOMOTIVE":"2013-01-02T00:00:00","19_BABY CARE":"2017-07-13T00:00:00","19_BEAUTY":"2013-01-02T00:00:00","19_BEVERAGES":"2013-01-02T00:00:00","19_BOOKS":"2013-01-01T00:00:00","19_BREAD/BAKERY":"2013-01-02T00:00:00","19_CELEBRATION":"2015-06-01T00:00:00","19_CLEANING":"2013-01-02T00:00:00","19_DAIRY":"2013-01-02T00:00:00","19_DELI":"2013-01-02T00:00:00","19_EGGS":"2013-01-02T00:00:00","19_FROZEN FOODS":"2013-01-02T00:00:00","19_GROCERY I":"2013-01-02T00:00:00","19_GROCERY II":"2013-01-02T00:00:00","19_HARDWARE":"2013-01-03T00:00:00","19_HOME AND KITCHEN I":"2014-07-01T00:00:00","19_HOME AND KITCHEN II":"2014-07-01T00:00:00","19_HOME APPLIANCES":"2017-02-02T00:00:00","19_HOME CARE":"2015-05-04T00:00:00","19_LADIESWEAR":"2015-06-01T00:00:00","19_LAWN AND GARDEN":"2017-09-01T00:00:00","19_LINGERIE":"2013-01-02T00:00:00","19_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","19_MAGAZINES":"2015-04-04T00:00:00","19_MEATS":"2013-01-02T00:00:00","19_PERSONAL CARE":"2013-01-02T00:00:00","19_PET SUPPLIES":"2015-06-01T00:00:00","19_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","19_POULTRY":"2013-01-02T00:00:00","19_PREPARED FOODS":"2013-01-02T00:00:00","19_PRODUCE":"2015-06-01T00:00:00","19_SCHOOL AND OFFICE SUPPLIES":"2015-01-01T00:00:00","19_SEAFOOD":"2014-08-22T00:00:00","2_AUTOMOTIVE":"2013-01-02T00:00:00","2_BABY CARE":"2017-09-01T00:00:00","2_BEAUTY":"2013-01-02T00:00:00","2_BEVERAGES":"2013-01-02T00:00:00","2_BOOKS":"2017-09-01T00:00:00","2_BREAD/BAKERY":"2013-01-02T00:00:00","2_CELEBRATION":"2015-06-01T00:00:00","2_CLEANING":"2013-01-02T00:00:00","2_DAIRY":"2013-01-02T00:00:00","2_DELI":"2013-01-02T00:00:00","2_EGGS":"2013-01-02T00:00:00","2_FROZEN FOODS":"2013-01-02T00:00:00","2_GROCERY I":"2013-01-02T00:00:00","2_GROCERY II":"2013-01-02T00:00:00","2_HARDWARE":"2013-01-03T00:00:00","2_HOME AND KITCHEN I":"2014-07-01T00:00:00","2_HOME AND KITCHEN II":"2014-07-01T00:00:00","2_HOME APPLIANCES":"2015-08-11T00:00:00","2_HOME CARE":"2015-05-04T00:00:00","2_LADIESWEAR":"2015-06-01T00:00:00","2_LAWN AND GARDEN":"2013-01-02T00:00:00","2_LINGERIE":"2013-01-02T00:00:00","2_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","2_MAGAZINES":"2015-04-01T00:00:00","2_MEATS":"2013-01-02T00:00:00","2_PERSONAL CARE":"2013-01-02T00:00:00","2_PET SUPPLIES":"2015-05-25T00:00:00","2_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","2_POULTRY":"2013-01-02T00:00:00","2_PREPARED FOODS":"2013-01-02T00:00:00","2_PRODUCE":"2013-03-16T00:00:00","2_SCHOOL AND OFFICE SUPPLIES":"2017-09-01T00:00:00","2_SEAFOOD":"2013-01-02T00:00:00","20_AUTOMOTIVE":"2015-02-13T00:00:00","20_BABY CARE":"2015-12-07T00:00:00","20_BEAUTY":"2015-02-13T00:00:00","20_BEVERAGES":"2015-02-13T00:00:00","20_BOOKS":"2013-01-01T00:00:00","20_BREAD/BAKERY":"2015-02-13T00:00:00","20_CELEBRATION":"2015-06-01T00:00:00","20_CLEANING":"2015-02-13T00:00:00","20_DAIRY":"2015-02-13T00:00:00","20_DELI":"2015-02-13T00:00:00","20_EGGS":"2015-02-13T00:00:00","20_FROZEN FOODS":"2015-02-13T00:00:00","20_GROCERY I":"2015-02-13T00:00:00","20_GROCERY II":"2015-02-13T00:00:00","20_HARDWARE":"2015-02-13T00:00:00","20_HOME AND KITCHEN I":"2015-02-13T00:00:00","20_HOME AND KITCHEN II":"2015-02-13T00:00:00","20_HOME APPLIANCES":"2017-09-01T00:00:00","20_HOME CARE":"2015-05-04T00:00:00","20_LADIESWEAR":"2015-06-01T00:00:00","20_LAWN AND GARDEN":"2015-02-13T00:00:00","20_LINGERIE":"2015-02-13T00:00:00","20_LIQUOR,WINE,BEER":"2015-02-13T00:00:00","20_MAGAZINES":"2015-04-10T00:00:00","20_MEATS":"2015-02-13T00:00:00","20_PERSONAL CARE":"2015-02-13T00:00:00","20_PET SUPPLIES":"2015-06-02T00:00:00","20_PLAYERS AND ELECTRONICS":"2015-06-02T00:00:00","20_POULTRY":"2015-02-13T00:00:00","20_PREPARED FOODS":"2015-02-13T00:00:00","20_PRODUCE":"2015-06-01T00:00:00","20_SCHOOL AND OFFICE SUPPLIES":"2015-06-04T00:00:00","20_SEAFOOD":"2015-02-13T00:00:00","21_AUTOMOTIVE":"2015-07-24T00:00:00","21_BABY CARE":"2015-12-06T00:00:00","21_BEAUTY":"2015-07-24T00:00:00","21_BEVERAGES":"2015-07-24T00:00:00","21_BOOKS":"2013-01-01T00:00:00","21_BREAD/BAKERY":"2015-07-24T00:00:00","21_CELEBRATION":"2015-07-24T00:00:00","21_CLEANING":"2015-07-24T00:00:00","21_DAIRY":"2015-07-24T00:00:00","21_DELI":"2015-07-24T00:00:00","21_EGGS":"2015-07-24T00:00:00","21_FROZEN FOODS":"2015-07-24T00:00:00","21_GROCERY I":"2015-07-24T00:00:00","21_GROCERY II":"2015-07-24T00:00:00","21_HARDWARE":"2015-07-24T00:00:00","21_HOME AND KITCHEN I":"2015-07-24T00:00:00","21_HOME AND KITCHEN II":"2015-07-24T00:00:00","21_HOME APPLIANCES":"2015-09-05T00:00:00","21_HOME CARE":"2015-07-24T00:00:00","21_LADIESWEAR":"2015-07-24T00:00:00","21_LAWN AND GARDEN":"2015-07-24T00:00:00","21_LINGERIE":"2015-07-24T00:00:00","21_LIQUOR,WINE,BEER":"2015-07-24T00:00:00","21_MAGAZINES":"2015-07-24T00:00:00","21_MEATS":"2015-07-24T00:00:00","21_PERSONAL CARE":"2015-07-24T00:00:00","21_PET SUPPLIES":"2015-07-24T00:00:00","21_PLAYERS AND ELECTRONICS":"2015-07-24T00:00:00","21_POULTRY":"2015-07-24T00:00:00","21_PREPARED FOODS":"2015-07-24T00:00:00","21_PRODUCE":"2015-07-24T00:00:00","21_SCHOOL AND OFFICE SUPPLIES":"2015-07-24T00:00:00","21_SEAFOOD":"2015-07-24T00:00:00","22_AUTOMOTIVE":"2015-10-09T00:00:00","22_BABY CARE":"2016-11-08T00:00:00","22_BEAUTY":"2015-10-09T00:00:00","22_BEVERAGES":"2015-10-09T00:00:00","22_BOOKS":"2013-01-01T00:00:00","22_BREAD/BAKERY":"2015-10-09T00:00:00","22_CELEBRATION":"2015-10-09T00:00:00","22_CLEANING":"2015-10-09T00:00:00","22_DAIRY":"2015-10-09T00:00:00","22_DELI":"2015-10-09T00:00:00","22_EGGS":"2015-10-09T00:00:00","22_FROZEN FOODS":"2015-10-09T00:00:00","22_GROCERY I":"2015-10-09T00:00:00","22_GROCERY II":"2015-10-09T00:00:00","22_HARDWARE":"2015-10-09T00:00:00","22_HOME AND KITCHEN I":"2015-10-09T00:00:00","22_HOME AND KITCHEN II":"2015-10-09T00:00:00","22_HOME APPLIANCES":"2017-09-01T00:00:00","22_HOME CARE":"2015-10-09T00:00:00","22_LADIESWEAR":"2017-04-20T00:00:00","22_LAWN AND GARDEN":"2017-09-01T00:00:00","22_LINGERIE":"2015-10-09T00:00:00","22_LIQUOR,WINE,BEER":"2015-10-09T00:00:00","22_MAGAZINES":"2015-10-09T00:00:00","22_MEATS":"2015-10-09T00:00:00","22_PERSONAL CARE":"2015-10-09T00:00:00","22_PET SUPPLIES":"2015-10-09T00:00:00","22_PLAYERS AND ELECTRONICS":"2015-10-09T00:00:00","22_POULTRY":"2015-10-09T00:00:00","22_PREPARED FOODS":"2015-10-09T00:00:00","22_PRODUCE":"2015-10-09T00:00:00","22_SCHOOL AND OFFICE SUPPLIES":"2017-03-17T00:00:00","22_SEAFOOD":"2015-10-09T00:00:00","23_AUTOMOTIVE":"2013-01-02T00:00:00","23_BABY CARE":"2013-01-01T00:00:00","23_BEAUTY":"2013-01-03T00:00:00","23_BEVERAGES":"2013-01-02T00:00:00","23_BOOKS":"2017-01-01T00:00:00","23_BREAD/BAKERY":"2013-01-02T00:00:00","23_CELEBRATION":"2015-06-01T00:00:00","23_CLEANING":"2013-01-02T00:00:00","23_DAIRY":"2013-01-02T00:00:00","23_DELI":"2013-01-02T00:00:00","23_EGGS":"2013-01-02T00:00:00","23_FROZEN FOODS":"2013-01-02T00:00:00","23_GROCERY I":"2013-01-02T00:00:00","23_GROCERY II":"2013-01-02T00:00:00","23_HARDWARE":"2016-05-09T00:00:00","23_HOME AND KITCHEN I":"2014-07-03T00:00:00","23_HOME AND KITCHEN II":"2014-07-01T00:00:00","23_HOME APPLIANCES":"2016-12-25T00:00:00","23_HOME CARE":"2015-05-04T00:00:00","23_LADIESWEAR":"2015-06-01T00:00:00","23_LAWN AND GARDEN":"2015-01-28T00:00:00","23_LINGERIE":"2013-01-02T00:00:00","23_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","23_MAGAZINES":"2015-04-07T00:00:00","23_MEATS":"2013-01-02T00:00:00","23_PERSONAL CARE":"2013-01-02T00:00:00","23_PET SUPPLIES":"2015-06-01T00:00:00","23_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","23_POULTRY":"2013-01-02T00:00:00","23_PREPARED FOODS":"2013-01-02T00:00:00","23_PRODUCE":"2013-03-16T00:00:00","23_SCHOOL AND OFFICE SUPPLIES":"2017-09-01T00:00:00","23_SEAFOOD":"2013-01-02T00:00:00","24_AUTOMOTIVE":"2014-07-24T00:00:00","24_BABY CARE":"2017-04-12T00:00:00","24_BEAUTY":"2014-07-24T00:00:00","24_BEVERAGES":"2014-07-24T00:00:00","24_BOOKS":"2017-09-01T00:00:00","24_BREAD/BAKERY":"2014-07-24T00:00:00","24_CELEBRATION":"2015-06-01T00:00:00","24_CLEANING":"2014-07-24T00:00:00","24_DAIRY":"2014-07-24T00:00:00","24_DELI":"2014-07-24T00:00:00","24_EGGS":"2014-07-24T00:00:00","24_FROZEN FOODS":"2014-07-24T00:00:00","24_GROCERY I":"2014-07-24T00:00:00","24_GROCERY II":"2014-07-24T00:00:00","24_HARDWARE":"2014-07-24T00:00:00","24_HOME AND KITCHEN I":"2014-07-24T00:00:00","24_HOME AND KITCHEN II":"2014-07-24T00:00:00","24_HOME APPLIANCES":"2017-09-01T00:00:00","24_HOME CARE":"2015-05-04T00:00:00","24_LADIESWEAR":"2015-06-01T00:00:00","24_LAWN AND GARDEN":"2014-07-24T00:00:00","24_LINGERIE":"2014-07-24T00:00:00","24_LIQUOR,WINE,BEER":"2014-07-24T00:00:00","24_MAGAZINES":"2015-04-01T00:00:00","24_MEATS":"2014-07-24T00:00:00","24_PERSONAL CARE":"2014-07-24T00:00:00","24_PET SUPPLIES":"2015-05-25T00:00:00","24_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","24_POULTRY":"2014-07-24T00:00:00","24_PREPARED FOODS":"2014-07-24T00:00:00","24_PRODUCE":"2014-07-24T00:00:00","24_SCHOOL AND OFFICE SUPPLIES":"2015-10-01T00:00:00","24_SEAFOOD":"2014-07-24T00:00:00","25_AUTOMOTIVE":"2016-10-27T00:00:00","25_BABY CARE":"2016-10-27T00:00:00","25_BEAUTY":"2016-10-27T00:00:00","25_BEVERAGES":"2016-10-27T00:00:00","25_BOOKS":"2016-10-15T00:00:00","25_BREAD/BAKERY":"2016-10-27T00:00:00","25_CELEBRATION":"2016-10-27T00:00:00","25_CLEANING":"2016-10-27T00:00:00","25_DAIRY":"2016-10-27T00:00:00","25_DELI":"2016-10-27T00:00:00","25_EGGS":"2016-10-27T00:00:00","25_FROZEN FOODS":"2016-10-27T00:00:00","25_GROCERY I":"2016-10-27T00:00:00","25_GROCERY II":"2016-10-27T00:00:00","25_HARDWARE":"2016-10-27T00:00:00","25_HOME AND KITCHEN I":"2016-10-27T00:00:00","25_HOME AND KITCHEN II":"2016-10-27T00:00:00","25_HOME APPLIANCES":"2016-12-21T00:00:00","25_HOME CARE":"2016-10-27T00:00:00","25_LADIESWEAR":"2013-01-01T00:00:00","25_LAWN AND GARDEN":"2016-10-27T00:00:00","25_LINGERIE":"2016-10-27T00:00:00","25_LIQUOR,WINE,BEER":"2016-10-27T00:00:00","25_MAGAZINES":"2016-10-27T00:00:00","25_MEATS":"2016-10-27T00:00:00","25_PERSONAL CARE":"2016-10-27T00:00:00","25_PET SUPPLIES":"2016-10-27T00:00:00","25_PLAYERS AND ELECTRONICS":"2016-10-27T00:00:00","25_POULTRY":"2016-10-27T00:00:00","25_PREPARED FOODS":"2016-10-27T00:00:00","25_PRODUCE":"2016-10-27T00:00:00","25_SCHOOL AND OFFICE SUPPLIES":"2016-10-27T00:00:00","25_SEAFOOD":"2016-10-27T00:00:00","26_AUTOMOTIVE":"2013-01-02T00:00:00","26_BABY CARE":"2017-09-01T00:00:00","26_BEAUTY":"2013-01-02T00:00:00","26_BEVERAGES":"2013-01-02T00:00:00","26_BOOKS":"2017-09-01T00:00:00","26_BREAD/BAKERY":"2013-01-02T00:00:00","26_CELEBRATION":"2015-06-01T00:00:00","26_CLEANING":"2013-01-02T00:00:00","26_DAIRY":"2013-01-02T00:00:00","26_DELI":"2013-01-02T00:00:00","26_EGGS":"2013-01-02T00:00:00","26_FROZEN FOODS":"2013-01-02T00:00:00","26_GROCERY I":"2013-01-02T00:00:00","26_GROCERY II":"2013-01-02T00:00:00","26_HARDWARE":"2013-01-02T00:00:00","26_HOME AND KITCHEN I":"2014-07-01T00:00:00","26_HOME AND KITCHEN II":"2014-07-01T00:00:00","26_HOME APPLIANCES":"2016-09-06T00:00:00","26_HOME CARE":"2015-05-04T00:00:00","26_LADIESWEAR":"2015-06-01T00:00:00","26_LAWN AND GARDEN":"2013-05-09T00:00:00","26_LINGERIE":"2013-01-02T00:00:00","26_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","26_MAGAZINES":"2015-04-12T00:00:00","26_MEATS":"2013-01-02T00:00:00","26_PERSONAL CARE":"2013-01-02T00:00:00","26_PET SUPPLIES":"2015-06-01T00:00:00","26_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","26_POULTRY":"2013-01-02T00:00:00","26_PREPARED FOODS":"2013-01-02T00:00:00","26_PRODUCE":"2013-03-16T00:00:00","26_SCHOOL AND OFFICE SUPPLIES":"2015-07-23T00:00:00","26_SEAFOOD":"2013-01-02T00:00:00","27_AUTOMOTIVE":"2013-01-02T00:00:00","27_BABY CARE":"2016-08-12T00:00:00","27_BEAUTY":"2013-01-02T00:00:00","27_BEVERAGES":"2013-01-02T00:00:00","27_BOOKS":"2016-10-14T00:00:00","27_BREAD/BAKERY":"2013-01-02T00:00:00","27_CELEBRATION":"2015-06-01T00:00:00","27_CLEANING":"2013-01-02T00:00:00","27_DAIRY":"2013-01-02T00:00:00","27_DELI":"2013-01-02T00:00:00","27_EGGS":"2013-01-02T00:00:00","27_FROZEN FOODS":"2013-01-02T00:00:00","27_GROCERY I":"2013-01-02T00:00:00","27_GROCERY II":"2013-01-02T00:00:00","27_HARDWARE":"2013-01-02T00:00:00","27_HOME AND KITCHEN I":"2014-07-01T00:00:00","27_HOME AND KITCHEN II":"2014-07-01T00:00:00","27_HOME APPLIANCES":"2016-12-17T00:00:00","27_HOME CARE":"2015-05-04T00:00:00","27_LADIESWEAR":"2015-06-01T00:00:00","27_LAWN AND GARDEN":"2013-01-02T00:00:00","27_LINGERIE":"2013-01-02T00:00:00","27_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","27_MAGAZINES":"2015-04-02T00:00:00","27_MEATS":"2013-01-02T00:00:00","27_PERSONAL CARE":"2013-01-02T00:00:00","27_PET SUPPLIES":"2015-05-26T00:00:00","27_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","27_POULTRY":"2013-01-02T00:00:00","27_PREPARED FOODS":"2013-01-02T00:00:00","27_PRODUCE":"2013-03-16T00:00:00","27_SCHOOL AND OFFICE SUPPLIES":"2016-03-10T00:00:00","27_SEAFOOD":"2013-01-02T00:00:00","28_AUTOMOTIVE":"2013-01-02T00:00:00","28_BABY CARE":"2017-09-01T00:00:00","28_BEAUTY":"2013-01-02T00:00:00","28_BEVERAGES":"2013-01-02T00:00:00","28_BOOKS":"2013-01-01T00:00:00","28_BREAD/BAKERY":"2013-01-02T00:00:00","28_CELEBRATION":"2015-06-01T00:00:00","28_CLEANING":"2013-01-02T00:00:00","28_DAIRY":"2013-01-02T00:00:00","28_DELI":"2013-01-02T00:00:00","28_EGGS":"2013-01-02T00:00:00","28_FROZEN FOODS":"2013-01-02T00:00:00","28_GROCERY I":"2013-01-02T00:00:00","28_GROCERY II":"2013-01-02T00:00:00","28_HARDWARE":"2013-01-04T00:00:00","28_HOME AND KITCHEN I":"2014-07-01T00:00:00","28_HOME AND KITCHEN II":"2014-07-01T00:00:00","28_HOME APPLIANCES":"2016-12-15T00:00:00","28_HOME CARE":"2015-05-04T00:00:00","28_LADIESWEAR":"2013-01-01T00:00:00","28_LAWN AND GARDEN":"2013-01-05T00:00:00","28_LINGERIE":"2013-01-02T00:00:00","28_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","28_MAGAZINES":"2015-04-20T00:00:00","28_MEATS":"2013-01-02T00:00:00","28_PERSONAL CARE":"2013-01-02T00:00:00","28_PET SUPPLIES":"2015-06-01T00:00:00","28_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","28_POULTRY":"2013-01-02T00:00:00","28_PREPARED FOODS":"2013-01-02T00:00:00","28_PRODUCE":"2015-06-01T00:00:00","28_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","28_SEAFOOD":"2013-01-02T00:00:00","29_AUTOMOTIVE":"2015-03-20T00:00:00","29_BABY CARE":"2017-05-23T00:00:00","29_BEAUTY":"2015-03-20T00:00:00","29_BEVERAGES":"2015-03-20T00:00:00","29_BOOKS":"2013-01-01T00:00:00","29_BREAD/BAKERY":"2015-03-20T00:00:00","29_CELEBRATION":"2015-06-01T00:00:00","29_CLEANING":"2015-03-20T00:00:00","29_DAIRY":"2015-03-20T00:00:00","29_DELI":"2015-03-20T00:00:00","29_EGGS":"2015-03-20T00:00:00","29_FROZEN FOODS":"2015-03-20T00:00:00","29_GROCERY I":"2015-03-20T00:00:00","29_GROCERY II":"2015-03-20T00:00:00","29_HARDWARE":"2015-03-20T00:00:00","29_HOME AND KITCHEN I":"2015-03-20T00:00:00","29_HOME AND KITCHEN II":"2015-03-20T00:00:00","29_HOME APPLIANCES":"2016-12-23T00:00:00","29_HOME CARE":"2015-05-04T00:00:00","29_LADIESWEAR":"2013-01-01T00:00:00","29_LAWN AND GARDEN":"2015-03-28T00:00:00","29_LINGERIE":"2015-03-20T00:00:00","29_LIQUOR,WINE,BEER":"2015-03-20T00:00:00","29_MAGAZINES":"2015-04-15T00:00:00","29_MEATS":"2015-03-20T00:00:00","29_PERSONAL CARE":"2015-03-20T00:00:00","29_PET SUPPLIES":"2015-06-01T00:00:00","29_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","29_POULTRY":"2015-03-20T00:00:00","29_PREPARED FOODS":"2015-03-20T00:00:00","29_PRODUCE":"2015-06-01T00:00:00","29_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","29_SEAFOOD":"2015-03-20T00:00:00","3_AUTOMOTIVE":"2013-01-02T00:00:00","3_BABY CARE":"2016-07-16T00:00:00","3_BEAUTY":"2013-01-02T00:00:00","3_BEVERAGES":"2013-01-02T00:00:00","3_BOOKS":"2017-09-01T00:00:00","3_BREAD/BAKERY":"2013-01-02T00:00:00","3_CELEBRATION":"2015-06-01T00:00:00","3_CLEANING":"2013-01-02T00:00:00","3_DAIRY":"2013-01-02T00:00:00","3_DELI":"2013-01-02T00:00:00","3_EGGS":"2013-01-02T00:00:00","3_FROZEN FOODS":"2013-01-02T00:00:00","3_GROCERY I":"2013-01-02T00:00:00","3_GROCERY II":"2013-01-02T00:00:00","3_HARDWARE":"2013-01-02T00:00:00","3_HOME AND KITCHEN I":"2014-07-01T00:00:00","3_HOME AND KITCHEN II":"2014-07-01T00:00:00","3_HOME APPLIANCES":"2015-08-06T00:00:00","3_HOME CARE":"2015-05-04T00:00:00","3_LADIESWEAR":"2015-06-01T00:00:00","3_LAWN AND GARDEN":"2013-01-02T00:00:00","3_LINGERIE":"2013-01-02T00:00:00","3_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","3_MAGAZINES":"2015-04-01T00:00:00","3_MEATS":"2013-01-02T00:00:00","3_PERSONAL CARE":"2013-01-02T00:00:00","3_PET SUPPLIES":"2015-05-25T00:00:00","3_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","3_POULTRY":"2013-01-02T00:00:00","3_PREPARED FOODS":"2013-01-02T00:00:00","3_PRODUCE":"2013-03-16T00:00:00","3_SCHOOL AND OFFICE SUPPLIES":"2015-11-11T00:00:00","3_SEAFOOD":"2013-01-02T00:00:00","30_AUTOMOTIVE":"2013-01-02T00:00:00","30_BABY CARE":"2017-05-09T00:00:00","30_BEAUTY":"2013-05-28T00:00:00","30_BEVERAGES":"2013-01-02T00:00:00","30_BOOKS":"2013-01-01T00:00:00","30_BREAD/BAKERY":"2013-01-02T00:00:00","30_CELEBRATION":"2015-06-01T00:00:00","30_CLEANING":"2013-01-02T00:00:00","30_DAIRY":"2013-01-02T00:00:00","30_DELI":"2013-01-02T00:00:00","30_EGGS":"2013-01-02T00:00:00","30_FROZEN FOODS":"2013-01-02T00:00:00","30_GROCERY I":"2013-01-02T00:00:00","30_GROCERY II":"2013-01-02T00:00:00","30_HARDWARE":"2013-01-02T00:00:00","30_HOME AND KITCHEN I":"2014-07-01T00:00:00","30_HOME AND KITCHEN II":"2014-07-01T00:00:00","30_HOME APPLIANCES":"2017-01-31T00:00:00","30_HOME CARE":"2015-05-04T00:00:00","30_LADIESWEAR":"2015-12-31T00:00:00","30_LAWN AND GARDEN":"2013-01-01T00:00:00","30_LINGERIE":"2013-01-02T00:00:00","30_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","30_MAGAZINES":"2015-04-13T00:00:00","30_MEATS":"2013-01-02T00:00:00","30_PERSONAL CARE":"2013-01-02T00:00:00","30_PET SUPPLIES":"2015-06-01T00:00:00","30_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","30_POULTRY":"2013-01-02T00:00:00","30_PREPARED FOODS":"2013-01-02T00:00:00","30_PRODUCE":"2015-06-01T00:00:00","30_SCHOOL AND OFFICE SUPPLIES":"2017-08-03T00:00:00","30_SEAFOOD":"2014-09-07T00:00:00","31_AUTOMOTIVE":"2013-01-02T00:00:00","31_BABY CARE":"2015-06-10T00:00:00","31_BEAUTY":"2013-01-02T00:00:00","31_BEVERAGES":"2013-01-02T00:00:00","31_BOOKS":"2013-01-01T00:00:00","31_BREAD/BAKERY":"2013-01-02T00:00:00","31_CELEBRATION":"2015-06-01T00:00:00","31_CLEANING":"2013-01-02T00:00:00","31_DAIRY":"2013-01-02T00:00:00","31_DELI":"2013-01-02T00:00:00","31_EGGS":"2013-01-02T00:00:00","31_FROZEN FOODS":"2013-01-02T00:00:00","31_GROCERY I":"2013-01-02T00:00:00","31_GROCERY II":"2013-01-02T00:00:00","31_HARDWARE":"2013-01-02T00:00:00","31_HOME AND KITCHEN I":"2014-07-01T00:00:00","31_HOME AND KITCHEN II":"2014-07-01T00:00:00","31_HOME APPLIANCES":"2016-12-19T00:00:00","31_HOME CARE":"2015-05-04T00:00:00","31_LADIESWEAR":"2015-06-01T00:00:00","31_LAWN AND GARDEN":"2013-01-02T00:00:00","31_LINGERIE":"2013-01-02T00:00:00","31_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","31_MAGAZINES":"2015-04-01T00:00:00","31_MEATS":"2013-01-02T00:00:00","31_PERSONAL CARE":"2013-01-02T00:00:00","31_PET SUPPLIES":"2015-06-02T00:00:00","31_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","31_POULTRY":"2013-01-02T00:00:00","31_PREPARED FOODS":"2013-01-02T00:00:00","31_PRODUCE":"2015-06-01T00:00:00","31_SCHOOL AND OFFICE SUPPLIES":"2015-06-11T00:00:00","31_SEAFOOD":"2013-01-02T00:00:00","32_AUTOMOTIVE":"2013-01-02T00:00:00","32_BABY CARE":"2017-09-01T00:00:00","32_BEAUTY":"2014-06-19T00:00:00","32_BEVERAGES":"2013-01-02T00:00:00","32_BOOKS":"2013-01-01T00:00:00","32_BREAD/BAKERY":"2013-01-02T00:00:00","32_CELEBRATION":"2015-06-01T00:00:00","32_CLEANING":"2013-01-02T00:00:00","32_DAIRY":"2013-01-02T00:00:00","32_DELI":"2013-01-02T00:00:00","32_EGGS":"2013-01-02T00:00:00","32_FROZEN FOODS":"2013-01-02T00:00:00","32_GROCERY I":"2013-01-02T00:00:00","32_GROCERY II":"2013-01-02T00:00:00","32_HARDWARE":"2013-01-02T00:00:00","32_HOME AND KITCHEN I":"2014-07-01T00:00:00","32_HOME AND KITCHEN II":"2014-07-02T00:00:00","32_HOME APPLIANCES":"2017-07-13T00:00:00","32_HOME CARE":"2015-05-04T00:00:00","32_LADIESWEAR":"2013-01-01T00:00:00","32_LAWN AND GARDEN":"2017-09-01T00:00:00","32_LINGERIE":"2013-01-02T00:00:00","32_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","32_MAGAZINES":"2015-08-15T00:00:00","32_MEATS":"2013-01-02T00:00:00","32_PERSONAL CARE":"2013-01-02T00:00:00","32_PET SUPPLIES":"2016-11-21T00:00:00","32_PLAYERS AND ELECTRONICS":"2015-06-04T00:00:00","32_POULTRY":"2013-01-02T00:00:00","32_PREPARED FOODS":"2013-01-02T00:00:00","32_PRODUCE":"2015-06-01T00:00:00","32_SCHOOL AND OFFICE SUPPLIES":"2017-09-01T00:00:00","32_SEAFOOD":"2015-02-08T00:00:00","33_AUTOMOTIVE":"2013-01-02T00:00:00","33_BABY CARE":"2017-09-01T00:00:00","33_BEAUTY":"2013-01-02T00:00:00","33_BEVERAGES":"2013-01-02T00:00:00","33_BOOKS":"2013-01-01T00:00:00","33_BREAD/BAKERY":"2013-01-02T00:00:00","33_CELEBRATION":"2015-06-01T00:00:00","33_CLEANING":"2013-01-02T00:00:00","33_DAIRY":"2013-01-02T00:00:00","33_DELI":"2013-01-02T00:00:00","33_EGGS":"2013-01-02T00:00:00","33_FROZEN FOODS":"2013-01-02T00:00:00","33_GROCERY I":"2013-01-02T00:00:00","33_GROCERY II":"2013-01-02T00:00:00","33_HARDWARE":"2013-01-05T00:00:00","33_HOME AND KITCHEN I":"2014-07-01T00:00:00","33_HOME AND KITCHEN II":"2014-07-01T00:00:00","33_HOME APPLIANCES":"2016-11-08T00:00:00","33_HOME CARE":"2015-05-04T00:00:00","33_LADIESWEAR":"2013-01-01T00:00:00","33_LAWN AND GARDEN":"2017-09-01T00:00:00","33_LINGERIE":"2013-01-02T00:00:00","33_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","33_MAGAZINES":"2015-04-01T00:00:00","33_MEATS":"2013-01-02T00:00:00","33_PERSONAL CARE":"2013-01-02T00:00:00","33_PET SUPPLIES":"2015-06-02T00:00:00","33_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","33_POULTRY":"2013-01-02T00:00:00","33_PREPARED FOODS":"2013-01-02T00:00:00","33_PRODUCE":"2015-06-01T00:00:00","33_SCHOOL AND OFFICE SUPPLIES":"2017-09-01T00:00:00","33_SEAFOOD":"2014-09-02T00:00:00","34_AUTOMOTIVE":"2013-01-02T00:00:00","34_BABY CARE":"2015-12-16T00:00:00","34_BEAUTY":"2013-01-02T00:00:00","34_BEVERAGES":"2013-01-02T00:00:00","34_BOOKS":"2013-01-01T00:00:00","34_BREAD/BAKERY":"2013-01-02T00:00:00","34_CELEBRATION":"2015-06-01T00:00:00","34_CLEANING":"2013-01-02T00:00:00","34_DAIRY":"2013-01-02T00:00:00","34_DELI":"2013-01-02T00:00:00","34_EGGS":"2013-01-02T00:00:00","34_FROZEN FOODS":"2013-01-02T00:00:00","34_GROCERY I":"2013-01-02T00:00:00","34_GROCERY II":"2013-01-02T00:00:00","34_HARDWARE":"2013-01-02T00:00:00","34_HOME AND KITCHEN I":"2014-07-01T00:00:00","34_HOME AND KITCHEN II":"2014-07-01T00:00:00","34_HOME APPLIANCES":"2016-12-21T00:00:00","34_HOME CARE":"2015-05-04T00:00:00","34_LADIESWEAR":"2015-06-01T00:00:00","34_LAWN AND GARDEN":"2013-01-12T00:00:00","34_LINGERIE":"2013-01-02T00:00:00","34_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","34_MAGAZINES":"2015-04-07T00:00:00","34_MEATS":"2013-01-02T00:00:00","34_PERSONAL CARE":"2013-01-02T00:00:00","34_PET SUPPLIES":"2015-06-01T00:00:00","34_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","34_POULTRY":"2013-01-03T00:00:00","34_PREPARED FOODS":"2013-01-02T00:00:00","34_PRODUCE":"2015-06-01T00:00:00","34_SCHOOL AND OFFICE SUPPLIES":"2015-06-03T00:00:00","34_SEAFOOD":"2013-01-03T00:00:00","35_AUTOMOTIVE":"2013-01-02T00:00:00","35_BABY CARE":"2017-09-01T00:00:00","35_BEAUTY":"2013-01-04T00:00:00","35_BEVERAGES":"2013-01-02T00:00:00","35_BOOKS":"2013-01-01T00:00:00","35_BREAD/BAKERY":"2013-01-02T00:00:00","35_CELEBRATION":"2015-06-01T00:00:00","35_CLEANING":"2013-01-02T00:00:00","35_DAIRY":"2013-01-02T00:00:00","35_DELI":"2013-01-02T00:00:00","35_EGGS":"2013-01-02T00:00:00","35_FROZEN FOODS":"2013-01-02T00:00:00","35_GROCERY I":"2013-01-02T00:00:00","35_GROCERY II":"2013-01-02T00:00:00","35_HARDWARE":"2013-01-03T00:00:00","35_HOME AND KITCHEN I":"2014-07-01T00:00:00","35_HOME AND KITCHEN II":"2014-07-01T00:00:00","35_HOME APPLIANCES":"2016-12-01T00:00:00","35_HOME CARE":"2015-05-04T00:00:00","35_LADIESWEAR":"2013-01-01T00:00:00","35_LAWN AND GARDEN":"2013-01-02T00:00:00","35_LINGERIE":"2013-01-02T00:00:00","35_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","35_MAGAZINES":"2015-04-24T00:00:00","35_MEATS":"2013-01-02T00:00:00","35_PERSONAL CARE":"2013-01-02T00:00:00","35_PET SUPPLIES":"2016-08-01T00:00:00","35_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","35_POULTRY":"2013-01-02T00:00:00","35_PREPARED FOODS":"2013-01-02T00:00:00","35_PRODUCE":"2015-06-01T00:00:00","35_SCHOOL AND OFFICE SUPPLIES":"2017-03-14T00:00:00","35_SEAFOOD":"2016-01-21T00:00:00","36_AUTOMOTIVE":"2013-05-09T00:00:00","36_BABY CARE":"2017-09-01T00:00:00","36_BEAUTY":"2013-05-09T00:00:00","36_BEVERAGES":"2013-05-09T00:00:00","36_BOOKS":"2013-01-01T00:00:00","36_BREAD/BAKERY":"2013-05-09T00:00:00","36_CELEBRATION":"2015-06-01T00:00:00","36_CLEANING":"2013-05-09T00:00:00","36_DAIRY":"2013-05-09T00:00:00","36_DELI":"2013-05-09T00:00:00","36_EGGS":"2013-05-09T00:00:00","36_FROZEN FOODS":"2013-05-09T00:00:00","36_GROCERY I":"2013-05-09T00:00:00","36_GROCERY II":"2013-05-09T00:00:00","36_HARDWARE":"2013-05-09T00:00:00","36_HOME AND KITCHEN I":"2014-07-01T00:00:00","36_HOME AND KITCHEN II":"2014-07-01T00:00:00","36_HOME APPLIANCES":"2016-12-29T00:00:00","36_HOME CARE":"2015-05-04T00:00:00","36_LADIESWEAR":"2017-09-01T00:00:00","36_LAWN AND GARDEN":"2013-05-13T00:00:00","36_LINGERIE":"2013-05-09T00:00:00","36_LIQUOR,WINE,BEER":"2013-05-09T00:00:00","36_MAGAZINES":"2015-04-04T00:00:00","36_MEATS":"2013-05-09T00:00:00","36_PERSONAL CARE":"2013-05-09T00:00:00","36_PET SUPPLIES":"2015-06-02T00:00:00","36_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","36_POULTRY":"2013-05-09T00:00:00","36_PREPARED FOODS":"2013-05-09T00:00:00","36_PRODUCE":"2015-06-01T00:00:00","36_SCHOOL AND OFFICE SUPPLIES":"2017-02-20T00:00:00","36_SEAFOOD":"2013-05-09T00:00:00","37_AUTOMOTIVE":"2013-01-02T00:00:00","37_BABY CARE":"2017-09-01T00:00:00","37_BEAUTY":"2013-01-02T00:00:00","37_BEVERAGES":"2013-01-02T00:00:00","37_BOOKS":"2017-09-01T00:00:00","37_BREAD/BAKERY":"2013-01-02T00:00:00","37_CELEBRATION":"2015-06-01T00:00:00","37_CLEANING":"2013-01-02T00:00:00","37_DAIRY":"2013-01-02T00:00:00","37_DELI":"2013-01-02T00:00:00","37_EGGS":"2013-01-02T00:00:00","37_FROZEN FOODS":"2013-01-02T00:00:00","37_GROCERY I":"2013-01-02T00:00:00","37_GROCERY II":"2013-01-02T00:00:00","37_HARDWARE":"2013-01-03T00:00:00","37_HOME AND KITCHEN I":"2014-07-01T00:00:00","37_HOME AND KITCHEN II":"2014-07-01T00:00:00","37_HOME APPLIANCES":"2015-08-11T00:00:00","37_HOME CARE":"2015-05-04T00:00:00","37_LADIESWEAR":"2015-06-01T00:00:00","37_LAWN AND GARDEN":"2013-01-02T00:00:00","37_LINGERIE":"2013-01-02T00:00:00","37_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","37_MAGAZINES":"2015-04-01T00:00:00","37_MEATS":"2013-01-02T00:00:00","37_PERSONAL CARE":"2013-01-02T00:00:00","37_PET SUPPLIES":"2015-05-29T00:00:00","37_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","37_POULTRY":"2013-01-02T00:00:00","37_PREPARED FOODS":"2013-01-02T00:00:00","37_PRODUCE":"2013-03-16T00:00:00","37_SCHOOL AND OFFICE SUPPLIES":"2017-09-01T00:00:00","37_SEAFOOD":"2013-01-02T00:00:00","38_AUTOMOTIVE":"2013-01-02T00:00:00","38_BABY CARE":"2017-09-01T00:00:00","38_BEAUTY":"2013-01-02T00:00:00","38_BEVERAGES":"2013-01-02T00:00:00","38_BOOKS":"2017-09-01T00:00:00","38_BREAD/BAKERY":"2013-01-02T00:00:00","38_CELEBRATION":"2015-06-01T00:00:00","38_CLEANING":"2013-01-02T00:00:00","38_DAIRY":"2013-01-02T00:00:00","38_DELI":"2013-01-02T00:00:00","38_EGGS":"2013-01-02T00:00:00","38_FROZEN FOODS":"2013-01-02T00:00:00","38_GROCERY I":"2013-01-02T00:00:00","38_GROCERY II":"2013-01-02T00:00:00","38_HARDWARE":"2013-01-02T00:00:00","38_HOME AND KITCHEN I":"2014-07-01T00:00:00","38_HOME AND KITCHEN II":"2014-07-01T00:00:00","38_HOME APPLIANCES":"2017-09-01T00:00:00","38_HOME CARE":"2015-05-04T00:00:00","38_LADIESWEAR":"2015-06-01T00:00:00","38_LAWN AND GARDEN":"2013-01-02T00:00:00","38_LINGERIE":"2013-01-02T00:00:00","38_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","38_MAGAZINES":"2015-04-01T00:00:00","38_MEATS":"2013-01-02T00:00:00","38_PERSONAL CARE":"2013-01-02T00:00:00","38_PET SUPPLIES":"2015-06-01T00:00:00","38_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","38_POULTRY":"2013-01-02T00:00:00","38_PREPARED FOODS":"2013-01-02T00:00:00","38_PRODUCE":"2013-03-16T00:00:00","38_SCHOOL AND OFFICE SUPPLIES":"2017-07-31T00:00:00","38_SEAFOOD":"2013-01-02T00:00:00","39_AUTOMOTIVE":"2013-01-02T00:00:00","39_BABY CARE":"2015-12-08T00:00:00","39_BEAUTY":"2013-01-02T00:00:00","39_BEVERAGES":"2013-01-02T00:00:00","39_BOOKS":"2013-01-01T00:00:00","39_BREAD/BAKERY":"2013-01-02T00:00:00","39_CELEBRATION":"2015-06-01T00:00:00","39_CLEANING":"2013-01-02T00:00:00","39_DAIRY":"2013-01-02T00:00:00","39_DELI":"2013-01-02T00:00:00","39_EGGS":"2013-01-02T00:00:00","39_FROZEN FOODS":"2013-01-02T00:00:00","39_GROCERY I":"2013-01-02T00:00:00","39_GROCERY II":"2013-01-02T00:00:00","39_HARDWARE":"2013-01-02T00:00:00","39_HOME AND KITCHEN I":"2014-07-01T00:00:00","39_HOME AND KITCHEN II":"2014-07-01T00:00:00","39_HOME APPLIANCES":"2016-04-27T00:00:00","39_HOME CARE":"2015-05-04T00:00:00","39_LADIESWEAR":"2015-06-01T00:00:00","39_LAWN AND GARDEN":"2013-01-02T00:00:00","39_LINGERIE":"2013-01-02T00:00:00","39_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","39_MAGAZINES":"2015-04-07T00:00:00","39_MEATS":"2013-01-02T00:00:00","39_PERSONAL CARE":"2013-01-02T00:00:00","39_PET SUPPLIES":"2015-06-01T00:00:00","39_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","39_POULTRY":"2013-01-02T00:00:00","39_PREPARED FOODS":"2013-01-02T00:00:00","39_PRODUCE":"2015-06-01T00:00:00","39_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","39_SEAFOOD":"2013-01-02T00:00:00","4_AUTOMOTIVE":"2013-01-02T00:00:00","4_BABY CARE":"2017-07-29T00:00:00","4_BEAUTY":"2013-01-02T00:00:00","4_BEVERAGES":"2013-01-02T00:00:00","4_BOOKS":"2016-10-11T00:00:00","4_BREAD/BAKERY":"2013-01-02T00:00:00","4_CELEBRATION":"2015-06-01T00:00:00","4_CLEANING":"2013-01-02T00:00:00","4_DAIRY":"2013-01-02T00:00:00","4_DELI":"2013-01-02T00:00:00","4_EGGS":"2013-01-02T00:00:00","4_FROZEN FOODS":"2013-01-02T00:00:00","4_GROCERY I":"2013-01-02T00:00:00","4_GROCERY II":"2013-01-02T00:00:00","4_HARDWARE":"2013-01-04T00:00:00","4_HOME AND KITCHEN I":"2014-07-01T00:00:00","4_HOME AND KITCHEN II":"2014-07-01T00:00:00","4_HOME APPLIANCES":"2016-12-16T00:00:00","4_HOME CARE":"2015-05-04T00:00:00","4_LADIESWEAR":"2015-06-01T00:00:00","4_LAWN AND GARDEN":"2013-01-02T00:00:00","4_LINGERIE":"2013-01-02T00:00:00","4_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","4_MAGAZINES":"2015-04-02T00:00:00","4_MEATS":"2013-01-02T00:00:00","4_PERSONAL CARE":"2013-01-02T00:00:00","4_PET SUPPLIES":"2015-06-01T00:00:00","4_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","4_POULTRY":"2013-01-02T00:00:00","4_PREPARED FOODS":"2013-01-02T00:00:00","4_PRODUCE":"2013-03-16T00:00:00","4_SCHOOL AND OFFICE SUPPLIES":"2015-10-25T00:00:00","4_SEAFOOD":"2013-01-02T00:00:00","40_AUTOMOTIVE":"2013-01-02T00:00:00","40_BABY CARE":"2017-06-12T00:00:00","40_BEAUTY":"2013-01-02T00:00:00","40_BEVERAGES":"2013-01-02T00:00:00","40_BOOKS":"2013-01-01T00:00:00","40_BREAD/BAKERY":"2013-01-02T00:00:00","40_CELEBRATION":"2015-06-01T00:00:00","40_CLEANING":"2013-01-02T00:00:00","40_DAIRY":"2013-01-02T00:00:00","40_DELI":"2013-01-02T00:00:00","40_EGGS":"2013-01-02T00:00:00","40_FROZEN FOODS":"2013-01-02T00:00:00","40_GROCERY I":"2013-01-02T00:00:00","40_GROCERY II":"2013-01-02T00:00:00","40_HARDWARE":"2013-01-08T00:00:00","40_HOME AND KITCHEN I":"2014-07-01T00:00:00","40_HOME AND KITCHEN II":"2014-07-01T00:00:00","40_HOME APPLIANCES":"2016-12-30T00:00:00","40_HOME CARE":"2015-05-04T00:00:00","40_LADIESWEAR":"2013-01-01T00:00:00","40_LAWN AND GARDEN":"2016-02-28T00:00:00","40_LINGERIE":"2013-01-04T00:00:00","40_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","40_MAGAZINES":"2015-04-03T00:00:00","40_MEATS":"2013-01-02T00:00:00","40_PERSONAL CARE":"2013-01-02T00:00:00","40_PET SUPPLIES":"2015-06-01T00:00:00","40_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","40_POULTRY":"2013-01-02T00:00:00","40_PREPARED FOODS":"2013-01-02T00:00:00","40_PRODUCE":"2015-06-01T00:00:00","40_SCHOOL AND OFFICE SUPPLIES":"2015-09-01T00:00:00","40_SEAFOOD":"2013-09-21T00:00:00","41_AUTOMOTIVE":"2013-01-02T00:00:00","41_BABY CARE":"2017-06-30T00:00:00","41_BEAUTY":"2013-01-02T00:00:00","41_BEVERAGES":"2013-01-02T00:00:00","41_BOOKS":"2017-09-01T00:00:00","41_BREAD/BAKERY":"2013-01-02T00:00:00","41_CELEBRATION":"2015-06-01T00:00:00","41_CLEANING":"2013-01-02T00:00:00","41_DAIRY":"2013-01-02T00:00:00","41_DELI":"2013-01-02T00:00:00","41_EGGS":"2013-01-02T00:00:00","41_FROZEN FOODS":"2013-01-02T00:00:00","41_GROCERY I":"2013-01-02T00:00:00","41_GROCERY II":"2013-01-02T00:00:00","41_HARDWARE":"2013-01-02T00:00:00","41_HOME AND KITCHEN I":"2014-07-01T00:00:00","41_HOME AND KITCHEN II":"2014-07-01T00:00:00","41_HOME APPLIANCES":"2017-02-09T00:00:00","41_HOME CARE":"2015-05-04T00:00:00","41_LADIESWEAR":"2015-06-01T00:00:00","41_LAWN AND GARDEN":"2013-01-02T00:00:00","41_LINGERIE":"2013-01-02T00:00:00","41_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","41_MAGAZINES":"2015-04-02T00:00:00","41_MEATS":"2013-01-02T00:00:00","41_PERSONAL CARE":"2013-01-02T00:00:00","41_PET SUPPLIES":"2015-05-25T00:00:00","41_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","41_POULTRY":"2013-01-02T00:00:00","41_PREPARED FOODS":"2013-01-02T00:00:00","41_PRODUCE":"2013-03-16T00:00:00","41_SCHOOL AND OFFICE SUPPLIES":"2017-04-01T00:00:00","41_SEAFOOD":"2013-01-02T00:00:00","42_AUTOMOTIVE":"2015-08-21T00:00:00","42_BABY CARE":"2017-09-01T00:00:00","42_BEAUTY":"2015-08-21T00:00:00","42_BEVERAGES":"2015-08-21T00:00:00","42_BOOKS":"2017-09-01T00:00:00","42_BREAD/BAKERY":"2015-08-21T00:00:00","42_CELEBRATION":"2015-08-21T00:00:00","42_CLEANING":"2015-08-21T00:00:00","42_DAIRY":"2015-08-21T00:00:00","42_DELI":"2015-08-21T00:00:00","42_EGGS":"2015-08-21T00:00:00","42_FROZEN FOODS":"2015-08-21T00:00:00","42_GROCERY I":"2015-08-21T00:00:00","42_GROCERY II":"2015-08-21T00:00:00","42_HARDWARE":"2015-08-21T00:00:00","42_HOME AND KITCHEN I":"2015-08-21T00:00:00","42_HOME AND KITCHEN II":"2015-08-21T00:00:00","42_HOME APPLIANCES":"2016-12-17T00:00:00","42_HOME CARE":"2015-08-21T00:00:00","42_LADIESWEAR":"2015-08-21T00:00:00","42_LAWN AND GARDEN":"2015-08-21T00:00:00","42_LINGERIE":"2015-08-21T00:00:00","42_LIQUOR,WINE,BEER":"2015-08-21T00:00:00","42_MAGAZINES":"2015-08-21T00:00:00","42_MEATS":"2015-08-21T00:00:00","42_PERSONAL CARE":"2015-08-21T00:00:00","42_PET SUPPLIES":"2015-08-21T00:00:00","42_PLAYERS AND ELECTRONICS":"2015-08-21T00:00:00","42_POULTRY":"2015-08-21T00:00:00","42_PREPARED FOODS":"2015-08-21T00:00:00","42_PRODUCE":"2015-08-21T00:00:00","42_SCHOOL AND OFFICE SUPPLIES":"2017-07-30T00:00:00","42_SEAFOOD":"2015-08-21T00:00:00","43_AUTOMOTIVE":"2013-01-02T00:00:00","43_BABY CARE":"2017-06-29T00:00:00","43_BEAUTY":"2013-01-02T00:00:00","43_BEVERAGES":"2013-01-02T00:00:00","43_BOOKS":"2013-01-01T00:00:00","43_BREAD/BAKERY":"2013-01-02T00:00:00","43_CELEBRATION":"2015-06-01T00:00:00","43_CLEANING":"2013-01-02T00:00:00","43_DAIRY":"2013-01-02T00:00:00","43_DELI":"2013-01-02T00:00:00","43_EGGS":"2013-01-02T00:00:00","43_FROZEN FOODS":"2013-01-02T00:00:00","43_GROCERY I":"2013-01-02T00:00:00","43_GROCERY II":"2013-01-02T00:00:00","43_HARDWARE":"2016-11-09T00:00:00","43_HOME AND KITCHEN I":"2014-07-01T00:00:00","43_HOME AND KITCHEN II":"2014-07-01T00:00:00","43_HOME APPLIANCES":"2017-07-06T00:00:00","43_HOME CARE":"2015-05-04T00:00:00","43_LADIESWEAR":"2013-01-01T00:00:00","43_LAWN AND GARDEN":"2016-11-27T00:00:00","43_LINGERIE":"2013-01-02T00:00:00","43_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","43_MAGAZINES":"2015-04-14T00:00:00","43_MEATS":"2013-01-02T00:00:00","43_PERSONAL CARE":"2013-01-02T00:00:00","43_PET SUPPLIES":"2015-06-02T00:00:00","43_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","43_POULTRY":"2013-01-02T00:00:00","43_PREPARED FOODS":"2013-01-02T00:00:00","43_PRODUCE":"2015-06-01T00:00:00","43_SCHOOL AND OFFICE SUPPLIES":"2016-10-13T00:00:00","43_SEAFOOD":"2014-09-03T00:00:00","44_AUTOMOTIVE":"2013-01-02T00:00:00","44_BABY CARE":"2013-01-01T00:00:00","44_BEAUTY":"2013-01-02T00:00:00","44_BEVERAGES":"2013-01-02T00:00:00","44_BOOKS":"2016-10-09T00:00:00","44_BREAD/BAKERY":"2013-01-02T00:00:00","44_CELEBRATION":"2015-06-01T00:00:00","44_CLEANING":"2013-01-02T00:00:00","44_DAIRY":"2013-01-02T00:00:00","44_DELI":"2013-01-02T00:00:00","44_EGGS":"2013-01-02T00:00:00","44_FROZEN FOODS":"2013-01-02T00:00:00","44_GROCERY I":"2013-01-02T00:00:00","44_GROCERY II":"2013-01-02T00:00:00","44_HARDWARE":"2013-01-02T00:00:00","44_HOME AND KITCHEN I":"2014-07-01T00:00:00","44_HOME AND KITCHEN II":"2014-07-01T00:00:00","44_HOME APPLIANCES":"2016-12-14T00:00:00","44_HOME CARE":"2015-05-04T00:00:00","44_LADIESWEAR":"2015-06-01T00:00:00","44_LAWN AND GARDEN":"2013-01-02T00:00:00","44_LINGERIE":"2013-01-02T00:00:00","44_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","44_MAGAZINES":"2015-04-01T00:00:00","44_MEATS":"2013-01-02T00:00:00","44_PERSONAL CARE":"2013-01-02T00:00:00","44_PET SUPPLIES":"2015-06-01T00:00:00","44_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","44_POULTRY":"2013-01-02T00:00:00","44_PREPARED FOODS":"2013-01-02T00:00:00","44_PRODUCE":"2013-03-16T00:00:00","44_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","44_SEAFOOD":"2013-01-02T00:00:00","45_AUTOMOTIVE":"2013-01-02T00:00:00","45_BABY CARE":"2013-01-01T00:00:00","45_BEAUTY":"2013-01-02T00:00:00","45_BEVERAGES":"2013-01-02T00:00:00","45_BOOKS":"2017-07-24T00:00:00","45_BREAD/BAKERY":"2013-01-02T00:00:00","45_CELEBRATION":"2015-06-01T00:00:00","45_CLEANING":"2013-01-02T00:00:00","45_DAIRY":"2013-01-02T00:00:00","45_DELI":"2013-01-02T00:00:00","45_EGGS":"2013-01-02T00:00:00","45_FROZEN FOODS":"2013-01-02T00:00:00","45_GROCERY I":"2013-01-02T00:00:00","45_GROCERY II":"2013-01-02T00:00:00","45_HARDWARE":"2013-01-03T00:00:00","45_HOME AND KITCHEN I":"2014-07-01T00:00:00","45_HOME AND KITCHEN II":"2014-07-01T00:00:00","45_HOME APPLIANCES":"2016-08-06T00:00:00","45_HOME CARE":"2015-05-04T00:00:00","45_LADIESWEAR":"2015-06-01T00:00:00","45_LAWN AND GARDEN":"2013-01-02T00:00:00","45_LINGERIE":"2013-01-02T00:00:00","45_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","45_MAGAZINES":"2015-04-01T00:00:00","45_MEATS":"2013-01-02T00:00:00","45_PERSONAL CARE":"2013-01-02T00:00:00","45_PET SUPPLIES":"2015-05-25T00:00:00","45_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","45_POULTRY":"2013-01-02T00:00:00","45_PREPARED FOODS":"2013-01-02T00:00:00","45_PRODUCE":"2013-03-16T00:00:00","45_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","45_SEAFOOD":"2013-01-02T00:00:00","46_AUTOMOTIVE":"2013-01-02T00:00:00","46_BABY CARE":"2013-01-01T00:00:00","46_BEAUTY":"2013-01-02T00:00:00","46_BEVERAGES":"2013-01-02T00:00:00","46_BOOKS":"2016-10-13T00:00:00","46_BREAD/BAKERY":"2013-01-02T00:00:00","46_CELEBRATION":"2015-06-01T00:00:00","46_CLEANING":"2013-01-02T00:00:00","46_DAIRY":"2013-01-02T00:00:00","46_DELI":"2013-01-02T00:00:00","46_EGGS":"2013-01-02T00:00:00","46_FROZEN FOODS":"2013-01-02T00:00:00","46_GROCERY I":"2013-01-02T00:00:00","46_GROCERY II":"2013-01-02T00:00:00","46_HARDWARE":"2013-01-04T00:00:00","46_HOME AND KITCHEN I":"2014-07-01T00:00:00","46_HOME AND KITCHEN II":"2014-07-01T00:00:00","46_HOME APPLIANCES":"2016-12-20T00:00:00","46_HOME CARE":"2015-05-04T00:00:00","46_LADIESWEAR":"2015-06-01T00:00:00","46_LAWN AND GARDEN":"2013-01-02T00:00:00","46_LINGERIE":"2013-01-02T00:00:00","46_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","46_MAGAZINES":"2015-04-06T00:00:00","46_MEATS":"2013-01-02T00:00:00","46_PERSONAL CARE":"2013-01-02T00:00:00","46_PET SUPPLIES":"2015-05-26T00:00:00","46_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","46_POULTRY":"2013-01-02T00:00:00","46_PREPARED FOODS":"2013-01-02T00:00:00","46_PRODUCE":"2013-03-16T00:00:00","46_SCHOOL AND OFFICE SUPPLIES":"2015-06-02T00:00:00","46_SEAFOOD":"2013-01-02T00:00:00","47_AUTOMOTIVE":"2013-01-02T00:00:00","47_BABY CARE":"2013-01-01T00:00:00","47_BEAUTY":"2013-01-02T00:00:00","47_BEVERAGES":"2013-01-02T00:00:00","47_BOOKS":"2016-10-12T00:00:00","47_BREAD/BAKERY":"2013-01-02T00:00:00","47_CELEBRATION":"2015-06-01T00:00:00","47_CLEANING":"2013-01-02T00:00:00","47_DAIRY":"2013-01-02T00:00:00","47_DELI":"2013-01-02T00:00:00","47_EGGS":"2013-01-02T00:00:00","47_FROZEN FOODS":"2013-01-02T00:00:00","47_GROCERY I":"2013-01-02T00:00:00","47_GROCERY II":"2013-01-02T00:00:00","47_HARDWARE":"2013-01-02T00:00:00","47_HOME AND KITCHEN I":"2014-07-01T00:00:00","47_HOME AND KITCHEN II":"2014-07-01T00:00:00","47_HOME APPLIANCES":"2015-08-06T00:00:00","47_HOME CARE":"2015-05-04T00:00:00","47_LADIESWEAR":"2015-06-01T00:00:00","47_LAWN AND GARDEN":"2013-01-02T00:00:00","47_LINGERIE":"2013-01-02T00:00:00","47_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","47_MAGAZINES":"2015-04-01T00:00:00","47_MEATS":"2013-01-02T00:00:00","47_PERSONAL CARE":"2013-01-02T00:00:00","47_PET SUPPLIES":"2015-06-01T00:00:00","47_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","47_POULTRY":"2013-01-02T00:00:00","47_PREPARED FOODS":"2013-01-02T00:00:00","47_PRODUCE":"2013-03-16T00:00:00","47_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","47_SEAFOOD":"2013-01-02T00:00:00","48_AUTOMOTIVE":"2013-01-02T00:00:00","48_BABY CARE":"2013-01-01T00:00:00","48_BEAUTY":"2013-01-02T00:00:00","48_BEVERAGES":"2013-01-02T00:00:00","48_BOOKS":"2017-09-01T00:00:00","48_BREAD/BAKERY":"2013-01-02T00:00:00","48_CELEBRATION":"2015-06-01T00:00:00","48_CLEANING":"2013-01-02T00:00:00","48_DAIRY":"2013-01-02T00:00:00","48_DELI":"2013-01-02T00:00:00","48_EGGS":"2013-01-02T00:00:00","48_FROZEN FOODS":"2013-01-02T00:00:00","48_GROCERY I":"2013-01-02T00:00:00","48_GROCERY II":"2013-01-02T00:00:00","48_HARDWARE":"2013-01-02T00:00:00","48_HOME AND KITCHEN I":"2014-07-01T00:00:00","48_HOME AND KITCHEN II":"2014-07-01T00:00:00","48_HOME APPLIANCES":"2017-01-22T00:00:00","48_HOME CARE":"2015-05-04T00:00:00","48_LADIESWEAR":"2015-06-01T00:00:00","48_LAWN AND GARDEN":"2013-01-02T00:00:00","48_LINGERIE":"2013-01-02T00:00:00","48_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","48_MAGAZINES":"2015-04-10T00:00:00","48_MEATS":"2013-01-02T00:00:00","48_PERSONAL CARE":"2013-01-02T00:00:00","48_PET SUPPLIES":"2015-05-27T00:00:00","48_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","48_POULTRY":"2013-01-02T00:00:00","48_PREPARED FOODS":"2013-01-02T00:00:00","48_PRODUCE":"2013-03-16T00:00:00","48_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","48_SEAFOOD":"2013-01-02T00:00:00","49_AUTOMOTIVE":"2013-01-02T00:00:00","49_BABY CARE":"2013-01-01T00:00:00","49_BEAUTY":"2013-01-02T00:00:00","49_BEVERAGES":"2013-01-02T00:00:00","49_BOOKS":"2017-09-01T00:00:00","49_BREAD/BAKERY":"2013-01-02T00:00:00","49_CELEBRATION":"2015-06-01T00:00:00","49_CLEANING":"2013-01-02T00:00:00","49_DAIRY":"2013-01-02T00:00:00","49_DELI":"2013-01-02T00:00:00","49_EGGS":"2013-01-02T00:00:00","49_FROZEN FOODS":"2013-01-02T00:00:00","49_GROCERY I":"2013-01-02T00:00:00","49_GROCERY II":"2013-01-02T00:00:00","49_HARDWARE":"2013-01-02T00:00:00","49_HOME AND KITCHEN I":"2014-07-01T00:00:00","49_HOME AND KITCHEN II":"2014-07-01T00:00:00","49_HOME APPLIANCES":"2016-11-17T00:00:00","49_HOME CARE":"2015-05-04T00:00:00","49_LADIESWEAR":"2015-06-01T00:00:00","49_LAWN AND GARDEN":"2013-01-04T00:00:00","49_LINGERIE":"2013-01-02T00:00:00","49_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","49_MAGAZINES":"2015-04-01T00:00:00","49_MEATS":"2013-01-02T00:00:00","49_PERSONAL CARE":"2013-01-02T00:00:00","49_PET SUPPLIES":"2015-05-25T00:00:00","49_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","49_POULTRY":"2013-01-02T00:00:00","49_PREPARED FOODS":"2013-01-02T00:00:00","49_PRODUCE":"2013-03-16T00:00:00","49_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","49_SEAFOOD":"2013-01-02T00:00:00","5_AUTOMOTIVE":"2013-01-02T00:00:00","5_BABY CARE":"2017-05-04T00:00:00","5_BEAUTY":"2013-01-02T00:00:00","5_BEVERAGES":"2013-01-02T00:00:00","5_BOOKS":"2017-09-01T00:00:00","5_BREAD/BAKERY":"2013-01-02T00:00:00","5_CELEBRATION":"2015-06-01T00:00:00","5_CLEANING":"2013-01-02T00:00:00","5_DAIRY":"2013-01-02T00:00:00","5_DELI":"2013-01-02T00:00:00","5_EGGS":"2013-01-02T00:00:00","5_FROZEN FOODS":"2013-01-02T00:00:00","5_GROCERY I":"2013-01-02T00:00:00","5_GROCERY II":"2013-01-02T00:00:00","5_HARDWARE":"2013-01-02T00:00:00","5_HOME AND KITCHEN I":"2014-07-01T00:00:00","5_HOME AND KITCHEN II":"2014-07-01T00:00:00","5_HOME APPLIANCES":"2016-12-15T00:00:00","5_HOME CARE":"2015-05-04T00:00:00","5_LADIESWEAR":"2015-06-01T00:00:00","5_LAWN AND GARDEN":"2013-01-04T00:00:00","5_LINGERIE":"2013-01-02T00:00:00","5_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","5_MAGAZINES":"2015-04-02T00:00:00","5_MEATS":"2013-01-02T00:00:00","5_PERSONAL CARE":"2013-01-02T00:00:00","5_PET SUPPLIES":"2015-06-01T00:00:00","5_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","5_POULTRY":"2013-01-02T00:00:00","5_PREPARED FOODS":"2013-01-02T00:00:00","5_PRODUCE":"2013-03-16T00:00:00","5_SCHOOL AND OFFICE SUPPLIES":"2015-12-14T00:00:00","5_SEAFOOD":"2013-01-02T00:00:00","50_AUTOMOTIVE":"2013-01-02T00:00:00","50_BABY CARE":"2013-01-01T00:00:00","50_BEAUTY":"2013-01-02T00:00:00","50_BEVERAGES":"2013-01-02T00:00:00","50_BOOKS":"2016-10-11T00:00:00","50_BREAD/BAKERY":"2013-01-02T00:00:00","50_CELEBRATION":"2015-06-01T00:00:00","50_CLEANING":"2013-01-02T00:00:00","50_DAIRY":"2013-01-02T00:00:00","50_DELI":"2013-01-02T00:00:00","50_EGGS":"2013-01-02T00:00:00","50_FROZEN FOODS":"2013-01-02T00:00:00","50_GROCERY I":"2013-01-02T00:00:00","50_GROCERY II":"2013-01-02T00:00:00","50_HARDWARE":"2013-01-03T00:00:00","50_HOME AND KITCHEN I":"2014-07-01T00:00:00","50_HOME AND KITCHEN II":"2014-07-01T00:00:00","50_HOME APPLIANCES":"2016-12-24T00:00:00","50_HOME CARE":"2015-05-04T00:00:00","50_LADIESWEAR":"2015-06-01T00:00:00","50_LAWN AND GARDEN":"2013-01-03T00:00:00","50_LINGERIE":"2013-01-02T00:00:00","50_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","50_MAGAZINES":"2015-04-01T00:00:00","50_MEATS":"2013-01-02T00:00:00","50_PERSONAL CARE":"2013-01-02T00:00:00","50_PET SUPPLIES":"2015-05-28T00:00:00","50_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","50_POULTRY":"2013-01-02T00:00:00","50_PREPARED FOODS":"2013-01-02T00:00:00","50_PRODUCE":"2013-03-16T00:00:00","50_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","50_SEAFOOD":"2013-01-02T00:00:00","51_AUTOMOTIVE":"2013-01-02T00:00:00","51_BABY CARE":"2013-01-01T00:00:00","51_BEAUTY":"2013-01-02T00:00:00","51_BEVERAGES":"2013-01-02T00:00:00","51_BOOKS":"2016-10-12T00:00:00","51_BREAD/BAKERY":"2013-01-02T00:00:00","51_CELEBRATION":"2015-06-01T00:00:00","51_CLEANING":"2013-01-02T00:00:00","51_DAIRY":"2013-01-02T00:00:00","51_DELI":"2013-01-02T00:00:00","51_EGGS":"2013-01-02T00:00:00","51_FROZEN FOODS":"2013-01-02T00:00:00","51_GROCERY I":"2013-01-02T00:00:00","51_GROCERY II":"2013-01-02T00:00:00","51_HARDWARE":"2013-01-02T00:00:00","51_HOME AND KITCHEN I":"2014-07-01T00:00:00","51_HOME AND KITCHEN II":"2014-07-01T00:00:00","51_HOME APPLIANCES":"2016-08-08T00:00:00","51_HOME CARE":"2015-05-04T00:00:00","51_LADIESWEAR":"2015-06-01T00:00:00","51_LAWN AND GARDEN":"2013-01-02T00:00:00","51_LINGERIE":"2013-01-02T00:00:00","51_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","51_MAGAZINES":"2015-04-01T00:00:00","51_MEATS":"2013-01-02T00:00:00","51_PERSONAL CARE":"2013-01-02T00:00:00","51_PET SUPPLIES":"2015-06-01T00:00:00","51_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","51_POULTRY":"2013-01-02T00:00:00","51_PREPARED FOODS":"2013-01-02T00:00:00","51_PRODUCE":"2013-03-16T00:00:00","51_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","51_SEAFOOD":"2013-01-02T00:00:00","52_AUTOMOTIVE":"2017-04-20T00:00:00","52_BABY CARE":"2013-01-01T00:00:00","52_BEAUTY":"2017-04-20T00:00:00","52_BEVERAGES":"2017-04-20T00:00:00","52_BOOKS":"2013-01-01T00:00:00","52_BREAD/BAKERY":"2017-04-20T00:00:00","52_CELEBRATION":"2017-04-20T00:00:00","52_CLEANING":"2017-04-20T00:00:00","52_DAIRY":"2017-04-20T00:00:00","52_DELI":"2017-04-20T00:00:00","52_EGGS":"2017-04-20T00:00:00","52_FROZEN FOODS":"2017-04-20T00:00:00","52_GROCERY I":"2017-04-20T00:00:00","52_GROCERY II":"2017-04-20T00:00:00","52_HARDWARE":"2017-04-20T00:00:00","52_HOME AND KITCHEN I":"2017-04-20T00:00:00","52_HOME AND KITCHEN II":"2017-04-20T00:00:00","52_HOME APPLIANCES":"2017-04-20T00:00:00","52_HOME CARE":"2017-04-20T00:00:00","52_LADIESWEAR":"2017-04-20T00:00:00","52_LAWN AND GARDEN":"2017-04-20T00:00:00","52_LINGERIE":"2017-04-20T00:00:00","52_LIQUOR,WINE,BEER":"2017-04-20T00:00:00","52_MAGAZINES":"2017-04-20T00:00:00","52_MEATS":"2017-04-20T00:00:00","52_PERSONAL CARE":"2017-04-20T00:00:00","52_PET SUPPLIES":"2017-04-20T00:00:00","52_PLAYERS AND ELECTRONICS":"2017-04-20T00:00:00","52_POULTRY":"2017-04-20T00:00:00","52_PREPARED FOODS":"2017-04-20T00:00:00","52_PRODUCE":"2017-04-20T00:00:00","52_SCHOOL AND OFFICE SUPPLIES":"2017-04-20T00:00:00","52_SEAFOOD":"2017-04-20T00:00:00","53_AUTOMOTIVE":"2014-05-29T00:00:00","53_BABY CARE":"2017-07-09T00:00:00","53_BEAUTY":"2014-05-29T00:00:00","53_BEVERAGES":"2014-05-29T00:00:00","53_BOOKS":"2015-10-01T00:00:00","53_BREAD/BAKERY":"2014-05-29T00:00:00","53_CELEBRATION":"2015-06-01T00:00:00","53_CLEANING":"2014-05-29T00:00:00","53_DAIRY":"2014-05-29T00:00:00","53_DELI":"2014-05-29T00:00:00","53_EGGS":"2014-05-29T00:00:00","53_FROZEN FOODS":"2014-05-29T00:00:00","53_GROCERY I":"2014-05-29T00:00:00","53_GROCERY II":"2014-05-29T00:00:00","53_HARDWARE":"2014-05-29T00:00:00","53_HOME AND KITCHEN I":"2014-07-01T00:00:00","53_HOME AND KITCHEN II":"2014-07-01T00:00:00","53_HOME APPLIANCES":"2016-12-16T00:00:00","53_HOME CARE":"2015-05-04T00:00:00","53_LADIESWEAR":"2015-06-01T00:00:00","53_LAWN AND GARDEN":"2014-05-29T00:00:00","53_LINGERIE":"2014-05-29T00:00:00","53_LIQUOR,WINE,BEER":"2014-05-29T00:00:00","53_MAGAZINES":"2015-04-05T00:00:00","53_MEATS":"2014-05-29T00:00:00","53_PERSONAL CARE":"2014-05-29T00:00:00","53_PET SUPPLIES":"2015-06-01T00:00:00","53_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","53_POULTRY":"2014-05-29T00:00:00","53_PREPARED FOODS":"2014-05-29T00:00:00","53_PRODUCE":"2014-05-29T00:00:00","53_SCHOOL AND OFFICE SUPPLIES":"2017-04-01T00:00:00","53_SEAFOOD":"2014-05-29T00:00:00","54_AUTOMOTIVE":"2013-01-02T00:00:00","54_BABY CARE":"2015-10-01T00:00:00","54_BEAUTY":"2013-01-05T00:00:00","54_BEVERAGES":"2013-01-02T00:00:00","54_BOOKS":"2013-01-01T00:00:00","54_BREAD/BAKERY":"2013-01-02T00:00:00","54_CELEBRATION":"2015-06-01T00:00:00","54_CLEANING":"2013-01-02T00:00:00","54_DAIRY":"2013-01-02T00:00:00","54_DELI":"2013-01-02T00:00:00","54_EGGS":"2013-01-02T00:00:00","54_FROZEN FOODS":"2013-01-02T00:00:00","54_GROCERY I":"2013-01-02T00:00:00","54_GROCERY II":"2013-01-02T00:00:00","54_HARDWARE":"2013-01-03T00:00:00","54_HOME AND KITCHEN I":"2014-07-01T00:00:00","54_HOME AND KITCHEN II":"2014-07-01T00:00:00","54_HOME APPLIANCES":"2016-12-23T00:00:00","54_HOME CARE":"2015-05-04T00:00:00","54_LADIESWEAR":"2013-01-01T00:00:00","54_LAWN AND GARDEN":"2013-01-01T00:00:00","54_LINGERIE":"2013-01-03T00:00:00","54_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","54_MAGAZINES":"2015-04-07T00:00:00","54_MEATS":"2013-01-02T00:00:00","54_PERSONAL CARE":"2013-01-02T00:00:00","54_PET SUPPLIES":"2017-06-17T00:00:00","54_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","54_POULTRY":"2013-01-02T00:00:00","54_PREPARED FOODS":"2013-01-02T00:00:00","54_PRODUCE":"2015-06-01T00:00:00","54_SCHOOL AND OFFICE SUPPLIES":"2017-03-15T00:00:00","54_SEAFOOD":"2013-10-12T00:00:00","6_AUTOMOTIVE":"2013-01-02T00:00:00","6_BABY CARE":"2017-07-10T00:00:00","6_BEAUTY":"2013-01-02T00:00:00","6_BEVERAGES":"2013-01-02T00:00:00","6_BOOKS":"2015-10-01T00:00:00","6_BREAD/BAKERY":"2013-01-02T00:00:00","6_CELEBRATION":"2015-06-01T00:00:00","6_CLEANING":"2013-01-02T00:00:00","6_DAIRY":"2013-01-02T00:00:00","6_DELI":"2013-01-02T00:00:00","6_EGGS":"2013-01-02T00:00:00","6_FROZEN FOODS":"2013-01-02T00:00:00","6_GROCERY I":"2013-01-02T00:00:00","6_GROCERY II":"2013-01-02T00:00:00","6_HARDWARE":"2014-06-28T00:00:00","6_HOME AND KITCHEN I":"2014-07-01T00:00:00","6_HOME AND KITCHEN II":"2014-07-01T00:00:00","6_HOME APPLIANCES":"2017-06-29T00:00:00","6_HOME CARE":"2015-05-04T00:00:00","6_LADIESWEAR":"2015-06-01T00:00:00","6_LAWN AND GARDEN":"2013-01-02T00:00:00","6_LINGERIE":"2013-01-02T00:00:00","6_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","6_MAGAZINES":"2015-04-04T00:00:00","6_MEATS":"2013-01-02T00:00:00","6_PERSONAL CARE":"2013-01-02T00:00:00","6_PET SUPPLIES":"2015-05-25T00:00:00","6_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","6_POULTRY":"2013-01-02T00:00:00","6_PREPARED FOODS":"2013-01-02T00:00:00","6_PRODUCE":"2013-03-16T00:00:00","6_SCHOOL AND OFFICE SUPPLIES":"2015-10-01T00:00:00","6_SEAFOOD":"2013-01-02T00:00:00","7_AUTOMOTIVE":"2013-01-02T00:00:00","7_BABY CARE":"2016-10-25T00:00:00","7_BEAUTY":"2013-01-02T00:00:00","7_BEVERAGES":"2013-01-02T00:00:00","7_BOOKS":"2016-10-11T00:00:00","7_BREAD/BAKERY":"2013-01-02T00:00:00","7_CELEBRATION":"2015-06-01T00:00:00","7_CLEANING":"2013-01-02T00:00:00","7_DAIRY":"2013-01-02T00:00:00","7_DELI":"2013-01-02T00:00:00","7_EGGS":"2013-01-02T00:00:00","7_FROZEN FOODS":"2013-01-02T00:00:00","7_GROCERY I":"2013-01-02T00:00:00","7_GROCERY II":"2013-01-02T00:00:00","7_HARDWARE":"2013-01-05T00:00:00","7_HOME AND KITCHEN I":"2014-07-01T00:00:00","7_HOME AND KITCHEN II":"2014-07-01T00:00:00","7_HOME APPLIANCES":"2016-12-14T00:00:00","7_HOME CARE":"2015-05-04T00:00:00","7_LADIESWEAR":"2015-06-01T00:00:00","7_LAWN AND GARDEN":"2013-01-02T00:00:00","7_LINGERIE":"2013-01-02T00:00:00","7_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","7_MAGAZINES":"2015-04-01T00:00:00","7_MEATS":"2013-01-02T00:00:00","7_PERSONAL CARE":"2013-01-02T00:00:00","7_PET SUPPLIES":"2015-05-27T00:00:00","7_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","7_POULTRY":"2013-01-02T00:00:00","7_PREPARED FOODS":"2013-01-02T00:00:00","7_PRODUCE":"2013-03-16T00:00:00","7_SCHOOL AND OFFICE SUPPLIES":"2015-07-25T00:00:00","7_SEAFOOD":"2013-01-02T00:00:00","8_AUTOMOTIVE":"2013-01-02T00:00:00","8_BABY CARE":"2016-07-12T00:00:00","8_BEAUTY":"2013-01-02T00:00:00","8_BEVERAGES":"2013-01-02T00:00:00","8_BOOKS":"2017-07-22T00:00:00","8_BREAD/BAKERY":"2013-01-02T00:00:00","8_CELEBRATION":"2015-06-01T00:00:00","8_CLEANING":"2013-01-02T00:00:00","8_DAIRY":"2013-01-02T00:00:00","8_DELI":"2013-01-02T00:00:00","8_EGGS":"2013-01-02T00:00:00","8_FROZEN FOODS":"2013-01-02T00:00:00","8_GROCERY I":"2013-01-02T00:00:00","8_GROCERY II":"2013-01-02T00:00:00","8_HARDWARE":"2013-01-03T00:00:00","8_HOME AND KITCHEN I":"2014-07-01T00:00:00","8_HOME AND KITCHEN II":"2014-07-01T00:00:00","8_HOME APPLIANCES":"2014-01-15T00:00:00","8_HOME CARE":"2015-05-04T00:00:00","8_LADIESWEAR":"2015-06-01T00:00:00","8_LAWN AND GARDEN":"2013-01-02T00:00:00","8_LINGERIE":"2013-01-02T00:00:00","8_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","8_MAGAZINES":"2015-04-06T00:00:00","8_MEATS":"2013-01-02T00:00:00","8_PERSONAL CARE":"2013-01-02T00:00:00","8_PET SUPPLIES":"2015-05-25T00:00:00","8_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","8_POULTRY":"2013-01-02T00:00:00","8_PREPARED FOODS":"2013-01-02T00:00:00","8_PRODUCE":"2013-03-16T00:00:00","8_SCHOOL AND OFFICE SUPPLIES":"2015-06-03T00:00:00","8_SEAFOOD":"2013-01-02T00:00:00","9_AUTOMOTIVE":"2013-01-02T00:00:00","9_BABY CARE":"2015-07-28T00:00:00","9_BEAUTY":"2013-01-02T00:00:00","9_BEVERAGES":"2013-01-02T00:00:00","9_BOOKS":"2013-01-01T00:00:00","9_BREAD/BAKERY":"2013-01-02T00:00:00","9_CELEBRATION":"2015-06-01T00:00:00","9_CLEANING":"2013-01-02T00:00:00","9_DAIRY":"2013-01-02T00:00:00","9_DELI":"2013-01-02T00:00:00","9_EGGS":"2013-01-02T00:00:00","9_FROZEN FOODS":"2013-01-02T00:00:00","9_GROCERY I":"2013-01-02T00:00:00","9_GROCERY II":"2013-01-02T00:00:00","9_HARDWARE":"2013-01-03T00:00:00","9_HOME AND KITCHEN I":"2014-07-01T00:00:00","9_HOME AND KITCHEN II":"2014-07-01T00:00:00","9_HOME APPLIANCES":"2014-01-15T00:00:00","9_HOME CARE":"2015-05-04T00:00:00","9_LADIESWEAR":"2015-06-01T00:00:00","9_LAWN AND GARDEN":"2013-01-04T00:00:00","9_LINGERIE":"2013-01-02T00:00:00","9_LIQUOR,WINE,BEER":"2013-01-02T00:00:00","9_MAGAZINES":"2015-04-20T00:00:00","9_MEATS":"2013-01-02T00:00:00","9_PERSONAL CARE":"2013-01-02T00:00:00","9_PET SUPPLIES":"2015-06-01T00:00:00","9_PLAYERS AND ELECTRONICS":"2015-06-01T00:00:00","9_POULTRY":"2013-01-02T00:00:00","9_PREPARED FOODS":"2013-01-02T00:00:00","9_PRODUCE":"2015-06-01T00:00:00","9_SCHOOL AND OFFICE SUPPLIES":"2015-06-01T00:00:00","9_SEAFOOD":"2013-01-02T00:00:00"}
    return reasonabledates
# Manualmente:
# 40_SCHOOL AND OFFICE SUPPLIES 2015-10-01
#'24_SCHOOL AND OFFICE SUPPLIES':'2015-10-01'
#54_SCHOOL AND OFFICE SUPPLIES':'2017-03-15'
#'25_BOOKS':'2016-10-15'
#'35_LAWN AND GARDEN':'2013-01-02'


In [9]:
'''
import json
def serialize(obj):
    if isinstance(obj, (pd.Timestamp, pd.Timestamp)):
        return obj.isoformat()  # Converti Timestamp in stringa
    raise TypeError(f"Type {type(obj)} not serializable")

# Convertiamo il dizionario in una stringa JSON densamente formattata
#dizionario_str = json.dumps(minimumDates, default=serialize, separators=(',', ':'))

# Stampa la stringa
#print(dizionario_str)

'''

'\nimport json\ndef serialize(obj):\n    if isinstance(obj, (pd.Timestamp, pd.Timestamp)):\n        return obj.isoformat()  # Converti Timestamp in stringa\n    raise TypeError(f"Type {type(obj)} not serializable")\n\n# Convertiamo il dizionario in una stringa JSON densamente formattata\n#dizionario_str = json.dumps(minimumDates, default=serialize, separators=(\',\', \':\'))\n\n# Stampa la stringa\n#print(dizionario_str)\n\n'

**STEP3 alternativo**

In [10]:
# Carico i dataset
X_sample_fourier, series = step2()
series = series.set_index(['store_nbr','family','date'])


step1: 0
step1: 1
step1 addChristmas for key: 9_SEAFOODgetDpFourier: index unique
getDpFourier: 0
set Dp Fourier for key_model: 52_LAWN AND GARDEN

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime


minimumDates = getreasonabledates()#getMinReasonableDates()


features_fourier = X_sample_fourier.columns
# Devo dividere il dataset in train e test
# Provo ad utilizzare un unico modello lineare, in caso di problemi con uno unico posso passare
# ad un modello per chiave store/family
# Per accedere ad uno specifico store/family occorre specificare la chiave
# X.loc[(1,'AUTOMOTIVE')]

# Mergio i dati del series con quelli di fourier
X_fourier = X_sample_fourier.join(series)
categoric_features = [col for col in X_fourier.columns if X_fourier[col].dtype=='object']
label_Encoder=labelEncodeFeatures(X_fourier, categoric_features)

# Separo TRAIN da TEST
test_slice = X_fourier['index']>3000887
df_test = X_fourier[test_slice]
df_train = X_fourier[~test_slice]
print(0)

# separiamo train e validation
# La separazione deve avvenire a livello di data e non a livello di indice ovviamente
dateValidation = pd.to_datetime('2017-08-01') # validiamo sulla prima quindicina di Agosto
X_train = df_train.loc[(slice(None), slice(None), df_train.index.get_level_values('date') < dateValidation)]
X_val = df_train.loc[(slice(None), slice(None), df_train.index.get_level_values('date') >= dateValidation)]

# Abbiamo visto che purtroppo un unico modello non approssima bene quindi devo creare un dizionario
# che associa allo store-family un modello lineare
linear_models={}

# Per il modello lineare passo solamente le features che servono a stabilire l'andamento sinusoidale
coppie_uniche = X_train.index.unique()
print(1)

# Creiamo un set per memorizzare le coppie uniche
unique_pairs = set()
# Iteriamo su tutte le coppie nell'indice
for store, family, _ in coppie_uniche:
    unique_pairs.add((store, family))
print(2)

# Convertiamo il set in una lista e stampiamo
for store, family in unique_pairs:
    key_model = str(store) + '_' + family
    
    # Alleno il modello solamente sullo store e family corrente
    #date_slice = minimumDates.get(key_model).strftime('%Y-%m-%d')
    date_value = datetime.strptime(minimumDates.get(key_model), '%Y-%m-%dT%H:%M:%S')

    if date_value > datetime(2017, 1, 1):
        date_value = datetime(2016, 1, 1)  

    date_slice = date_value.strftime('%Y-%m-%d')

    
    current_slice = ((X_train.index.get_level_values('store_nbr')==store) &
                    (X_train.index.get_level_values('family')==family) &
                    (X_train.index.get_level_values('date') >= date_slice))
    current_slice_val = ((X_val.index.get_level_values('store_nbr')==store) &
                    (X_val.index.get_level_values('family')==family) &
                    (X_val.index.get_level_values('date') >= date_slice))
    
    if date_slice is not None:# Se la data_slice è nan vuol dire che non ci sono dati per allenare il modello
        
        y_train = X_train.loc[current_slice].pop('sales')
        y_val = X_val.loc[current_slice_val].pop('sales')

        model = LinearRegression(fit_intercept=True)
        model.fit(X_train.loc[current_slice][features_fourier], y_train)

        y_pred = model.predict(X_val.loc[current_slice_val][features_fourier])
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        #print("Mean Squared Error:", mse)

        linear_models[key_model] = model
        print('\r' + f'key_model: {key_model}, mse:{mse}, r2:{r2}', end='', flush=True)




0
1
2
key_model: 52_LAWN AND GARDEN, mse:345.6727581615605, r2:-0.17860843440447227

**Verifica dei modelli lineari, sembrano approssimare bene**

Creo il unperioded

In [12]:
#unique_stores = X_sample_fourier.index.get_level_values('store_nbr').unique()
#unique_families = X_sample_fourier.index.get_level_values('family').unique()
# Inizializzo un dataframe che contenga i valori predetti
predicted_values = pd.Series(index=X_sample_fourier.index)


coppie_uniche = X_sample_fourier.index.unique()
unique_pairs = set()
for store, family, _ in coppie_uniche:
    unique_pairs.add((store, family))
for store, family in unique_pairs:
    key_model = str(store) + '_' + family
    print('\r' + f'key_model: {key_model}', end='', flush=True)
    model_fourier = linear_models[key_model]
    filtered_data = X_sample_fourier.loc[(store, family)]
    predictions = model_fourier.predict(filtered_data)
    predicted_values.loc[(store, family)] = predictions

X_unperioded = X_sample_fourier.copy()
X_unperioded['predicted_values'] = predicted_values       
        
X_unperioded = X_unperioded.join(series)
# A questo punto i dati sinusoidali non ci interessano più? Per ora manteniamo
# Ripristino l'indice con il quale andranno effettuate le previsioni
X_unperioded = X_unperioded.reset_index()
X_unperioded = X_unperioded.set_index('index')

key_model: 52_LAWN AND GARDEN

Ora devo eseguire la join tra X_unperioded e series

# XGBoost

In [13]:
# Separo subito i dati di test da quelli utilizzati per allenare l'ultimo stage
X_unperioded.reset_index()
#X_unperioded=X_unperioded.sort_values(by='index')
#X_unperioded=X_unperioded.set_index('index')

testSlice = 3000888

#X_unperioded['dcoilwtico'] = fill_nan_with_nearest(X_unperioded['dcoilwtico'])
X_unperioded['unperioded'] = X_unperioded['sales'] - X_unperioded['predicted_values']
X_unperioded = X_unperioded.drop('predicted_values', axis=1)
X_unperioded = X_unperioded.drop('sales', axis=1)
#X_unperioded = X_unperioded.drop('date', axis=1)
label_encoder=labelEncodeFeatures(X_unperioded, ['type','family']) # Prendo il dizionario delle family


X = X_unperioded[X_unperioded.index.get_level_values('index')<testSlice].copy()
X_test = X_unperioded[X_unperioded.index.get_level_values('index')>=testSlice].copy()
_ = X_test.pop('unperioded') 

E' evidente che alcuni prodotti sono stati introdotti nella catena in momenti differenti quindi non ha senso allenare su tutta la serie temporale.
Eleggiamo una data di inizio per ogni prodotto
Stesso discorso per i negozi, bisogna eleggere una data di inizio attività

In [14]:
import numpy as np # linear algebra
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
'''
boost_model = xgb.XGBRegressor()
param_grid = {
    'n_estimators': [100, 130, 150],
    'learning_rate': [0.4, 0.5],
    'max_depth': [6, 7, 8],
}

# Imposta la ricerca a griglia
grid_search = GridSearchCV(estimator=boost_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)

# Avvia la ricerca
grid_search.fit(X_train, y_train)

# Visualizza i migliori parametri
print("Migliori parametri:", grid_search.best_params_)
Migliori parametri: {'learning_rate': 0.4, 'max_depth': 8, 'n_estimators': 150}
'''
modelBoosted={}

couples = X.groupby(['store_nbr','family']).count()

coppie_uniche = couples.index.unique()
unique_pairs = set()
for store, family_int in coppie_uniche:
    unique_pairs.add((store, family_int))
for store, family_int in unique_pairs:
    # Devo decodificare la family
    family = label_encoder.inverse_transform([family_int])[0]
    key_model = f'{store}_{family}'

    # Alleno il modello solamente sullo store e family corrente
    current_slice = (store, family)
    date_slice = minimumDates[key_model]

    date_value = datetime.strptime(minimumDates.get(key_model), '%Y-%m-%dT%H:%M:%S')

    if date_value > datetime(2017, 1, 1):
        date_value = datetime(2016, 1, 1)  

    date_slice = date_value.strftime('%Y-%m-%d')

    current_slice = ((X['store_nbr']==store) &
                    (X['family']==family_int) &
                    (X['date'] >= pd.to_datetime(date_slice)))
    
    XX = X.loc[current_slice].drop('date', axis=1)
    # Separo train da validation
    y = XX.pop('unperioded') 
    X_train, X_val, y_train, y_val = train_test_split(XX, y, test_size=0.2, random_state=42)
    boost_model = xgb.XGBRegressor(learning_rate=0.4, max_depth=8, n_estimators=150)
    cv_scores = cross_val_score(boost_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    media = np.mean(cv_scores)
    dev_std = np.std(cv_scores)
    print('\r' + f'key_model: {key_model}. Media:{media}, varianza:{dev_std}', end='', flush=True)
    
    boost_model.fit(X_train, y_train)

    modelBoosted[key_model] = boost_model


'''
cv_scores = cross_val_score(boost_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
mse_scores = -cv_scores
rmse_scores = np.sqrt(mse_scores)
print("RMSE medio dalla validazione incrociata:", rmse_scores.mean())

boost_model.fit(X_train, y_train)
y_pred = boost_model.predict(X_val)

test_rmse = np.sqrt(mean_squared_error(y_pred, y_val))
print("RMSE sul set di test:", test_rmse)
'''

key_model: 7_DAIRY. Media:-53015.40115433953, varianza:16613.77646044361

'\ncv_scores = cross_val_score(boost_model, X_train, y_train, cv=5, scoring=\'neg_mean_squared_error\')\nmse_scores = -cv_scores\nrmse_scores = np.sqrt(mse_scores)\nprint("RMSE medio dalla validazione incrociata:", rmse_scores.mean())\n\nboost_model.fit(X_train, y_train)\ny_pred = boost_model.predict(X_val)\n\ntest_rmse = np.sqrt(mean_squared_error(y_pred, y_val))\nprint("RMSE sul set di test:", test_rmse)\n'

In [15]:
X_join = pd.concat([X, X_test])
X_join = X_join.sort_values(by=['store_nbr','family'])

X_decode = pd.concat([X, X_test])
X_decode['family'] = label_encoder.inverse_transform(X_decode['family'])
X_decode = X_decode.reset_index()
pred_lev1 = pd.DataFrame({'level1_pred':predicted_values})
pred_lev1 = pred_lev1.reset_index()
y_final = pd.merge(pred_lev1, X_decode[['store_nbr', 'family', 'date', 'index']], on=['store_nbr', 'family', 'date'], how='left')

y_final = y_final.set_index(['store_nbr', 'family', 'date'])

# Predico in base allo store-family
# Per eseguire la predizione devo utilizzare il modello corretto
# Itero sulla X_join
y_pred_residuals = pd.Series(index=y_final.index)
# creo la corrispondenza store family date con id
couples = X_join.groupby(['store_nbr','family']).count()

coppie_uniche = couples.index.unique()
unique_pairs = set()
for store, family_int in coppie_uniche:
    unique_pairs.add((store, family_int))
for store, family_int in unique_pairs:
    # Devo decodificare la family
    family = label_encoder.inverse_transform([family_int])[0]
    key_model = f'{store}_{family}'
    boost_model = modelBoosted[key_model]
    # Eseguo le predizioni solamente per la parte interessata
    current_slice = ((X_join['store_nbr']==store) &
                    (X_join['family']==family_int))
    
    
    X_to_predict = X_join.loc[current_slice].drop(['date', 'unperioded'], axis=1)

    residuals_pred = boost_model.predict(X_to_predict)
    y_pred_residuals.loc[(store, family)] = residuals_pred

    print('\r' + f'residuals for model: {key_model}', end='', flush=True)

    
#predicted_values # Sono i valori predetti nel primo stage su una series che ha chiave multipla store/family/data
#len(predicted_values)#3036528 --> ordinati su store-family-date
#len(y_pred_residuals)#3036528 --> ordinati su store-family-date
#Bisogna a questo punto sommarli sugli stessi indici
#y_pred_residuals sono su indice id
# predicted_values sono su indice store, family, date
# A questo punto posso mappare direttamente su test e train mergiando

# vediamo come sommare
y_final = pd.DataFrame({'1_step':predicted_values, '2_step':y_pred_residuals})
y_final['SUM'] = y_final['1_step'] + y_final['2_step']
y_final = y_final.reset_index()

# Occorre ora ripristinare l'indice in base alla chiave
# In realtà mi interessa solamente l'indice dei dati di test
train, test = getTrainTest()
test = test.reset_index()
y_final = pd.merge(y_final, test[['store_nbr', 'family', 'date', 'id']], on=['store_nbr', 'family', 'date'], how='left')
#Seleziono solamente i dati di test da inviare
Y = y_final.loc[y_final['id'].isna()==False][['id', 'SUM']]
Y['id'] = Y['id'].astype('int64')
Y = Y.rename(columns={'SUM': 'sales'})


residuals for model: 7_DAIRY

Ora occorre unire i predicted_values del punto precedente ai residuals

In [16]:
csv_file_path = 'submission.csv'
Y.to_csv(csv_file_path, index=False, header=True)

In [17]:
Q = Y.copy() 
#Y['sales'].map(lambda x: 0 if x<0 else x)
Q['sales'] = Q['sales'].map(lambda x: 0 if x<0 else x)
Q[Q['sales']<0]
Q.to_csv(csv_file_path, index=False, header=True)